# [Manual step] Download CSV files from PDB containing information about those proteins whose annotation type is mpstruc
See the workflow PPT for details.

# [Manual step] Create a new tab in the CSV file which contains only the unique PDB IDs

# Separate the unique PDB IDs in batches of 1000
See the workflow PPT for rationale.

In [98]:
%reset -f

In [ ]:
import pandas as pd

# df1 = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\all mpstruc entries taken from PDB via advanced search.xlsx", sheet_name = "Unique PDB IDs")
df1 = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\all mpstruc entries taken from PDB via advanced search.xlsx", sheet_name = "Entity entity IDs")
# list_ids_unique = df1["PDB IDs"]
list_ids_unique = df1["Entity ID"]

# print(len(list_ids_unique))

############################################################################################################################################################################################################
# Calculate the biggest possible number of "len(list_ids_unique)" which can be completely divided by 1000 e.g., if "len(list_ids_unique)" equals 5200, then the biggest possible number will be 5000:
############################################################################################################################################################################################################

batch_size  = 1000                                  #it is set to 1000 because as on 20220801, PDB does not allows downloading more than 1000 FASTA seqs at a time
remainder   = len(list_ids_unique) % batch_size #suppose if "batch_size" equals 1000 & "len(list_ids_unique)" equals 5200, then "remainder" will be equal to 200
upper_limit = len(list_ids_unique) - remainder  #"upper_limit" will be the biggest possible number of "len(list_ids_unique)" which is divisible by 1000; in the above example it will be equal to 5000
# print(remainder)
# print(upper_limit)

#################################################
#Create the batches:
#################################################

list_temporary  = list()
list_of_lists   = list()
ctr = 0

for i in list_ids_unique[0:upper_limit + 1]: #taking the above example, the loop will run from 1st element to the 5000th element
    ctr += 1
    if ctr <= batch_size:
        list_temporary.append(i)
    if ctr > batch_size:
        ctr = 1
        list_of_lists.append(list_temporary)
        list_temporary = list()
        list_temporary.append(i)

#################################################
#Create the final batch:
#################################################      

list_temporary = list()
for i in list_ids_unique[upper_limit:]:
    list_temporary.append(i)
list_of_lists.append(list_temporary)

# print(len(list_of_lists))

#################################################
#Write the batches to text files:
#################################################   

ctr = 0
list_combined = list()
for sublist in list_of_lists:
    ctr += 1
    with open("D:\\" + "batch" + str(ctr) + ".txt", "a") as file2:
        for pdb_id in sublist:
            file2.write(pdb_id + " ")
            list_combined.append(pdb_id)
    file2.close()

# [Partially manual step] Paste the PDB IDs of each batch in the input box of https://www.rcsb.org/downloads and identify problematic IDs & download PDB IDs

When you will copy the PDB IDs of a given batch to the batch download input box of the PDB website, select the data file format as "PDB" & click on the "Generate file batches for download" button, it will show you the number of files that were found & the names of PDB IDs that were not found. The reasons for a given query PDB ID not being found are:

PDB ID has not been released or held from release e.g., 7PJ5.
PDB ID is not found on PDB e.g., 72LR.
PDB ID is obsolete & has been superseded by an updated ID e.g., 1UM3. It could also be possible that two obsolete PDB IDs redirect to a common superseding PDB ID e.g., 3BZ1 & 3BZ2 both are superseded by 4V62.
However you will notice that on the batch downloads page, the sum of IDs that were found and the IDs that were not found does not necessarily equals the total number of input PDB IDs. This is because some PDB IDs are not available in ".pdb" format but in ".cif" format. However, The server does not shows them as IDs that were not found. It simply excludes them from the batch download links. This problem will be solved at the very last stage. To address problems 1 and 2, save such IDs in a text file & then delete such IDs from the source text file. To address problem 3, save such IDs in a text file & replace the obsolete IDs with the superseded IDs in the source text file. If two obsolete PDB IDs redirect to a common superseding PDB ID, then delete one of the two obsolete PDB IDs in the source text file & replace the other one with the superseded ID. However, note that an obsolete superseding PDB ID may be available in ".pdb" format or ".cif" format. We will ignore this sub-problem at this stage & address it at a later stage.

Once problem 1,2,3 are solved, re-check for duplicates in the soruce text file (I indeed found duplicate IDs in it; could have been introduced due to manual correction), create a new source file & then re-create the batch text files from it (code to do this is given below). Again run the batches through the batch download page. This time the server should not be showing any PDB ID that was not found but still, the number of IDs found may not necessarily equal the total number of input PDB IDs due to unavailability in ".pdb" format. To solve this final problem, I could not find an efficient way. The only way I could think of was to download all the PDB files of that batch & compare their names with the PDB IDs in the text file of that batch & make a list of the PDB IDs whose corresponding files were not found (code to do this is given below). These would be the files whose structures are not available in ".pdb" format. Download them separately. Do this for every batch.

Note that the bash script provided by PDB server can also be used to download file but it has followig limitations:

Downloading via bash script is slow.
Some PDB IDs are not available as ".pdb" files but as ".cif" files due to which download fails for such IDs & there is no way to save such IDs to another text file.
The script will not help you identify & solve the problems mentioned in the last step.

##  Identify the PDB files that didn't get downloaded in each batch

In [80]:
import os
downloaded_batch_of_files = "D:\\batch-download-structures-1679640141168\\"
file_names = [i[3:7].upper() for i in os.listdir(downloaded_batch_of_files)]
print("total pdb files:", len(file_names))

batch_text_file_ids = "D:\\batch3.txt"
with open(batch_text_file_ids) as file1:
    str1 = file1.read()
    batch_ids = str1.split(" ")[0:-1]
    file1.close()

print("total pdb ids in batch file:", len(batch_ids))
print(set(batch_ids).difference(set(file_names)))

total pdb files: 848
total pdb ids in batch file: 881
{'7RF2', '6ZA8', '6YVR', '8A60', '6W1V', '7PP1', '7A0G', '6Z8N', '7RF5', '6YY5', '6W1P', '6Z2N', '6Z4Q', '7RF6', '6ZZ5', '7RF1', '6Z33', '6W1Q', '6ZIN', '6W1T', '7RF7', '6UWI', '6W1U', '7RF4', '6ZZH', '7Q7Q', '7PJ5', '7Q7P', '6W1R', '6Z4V', '7OBW', '7QIJ', '6W1O'}


# Remove the prefix "pdb" from the downloaded PDB files and then change the extension of ".ent" files to ".pdb" 

In [78]:
%reset -f

In [79]:
import os
import re

path = "D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_pdb\\all pdb files\\"
for file in os.listdir(path):
    name_old = path + file
    match1 = re.search(r"(pdb)(.+)(\.ent)", file)
    match2 = re.search(r"(pdb)(.+)(\.cif)", file)
    if match1:
        name_new = path + match1.group(2) + ".pdb"
        os.rename(name_old, name_new)
    if match2:
        name_new = path + match2.group(2) + match2.group(3)
        os.rename(name_old, name_new)

# Add hydrogens to downloaded PDB files using the molprobity
Use molprobity server to add hydrogens to CIF files as the offline version fails to parse them

## For WSL Ubuntu

In [1]:
%reset -f

In [ ]:
import os
import subprocess

path_input_files  = "D:\\files_pdb\\"
path_input_files2 = "/mnt/d/files_pdb/"
path_output_files = "/mnt/d/files_molprobity/"
path_reduce       = "/home/prathvi/MolProbity-master/bin/linux/reduce" #WSl shell somehow doesn't recognizes the "reduce" command even after adding its alias in the bashrc so I will provide the full path of "reduce" file

for pdb_file in os.listdir(path_input_files):
    print("processing", pdb_file)
    output_filename  = pdb_file[0:len(pdb_file)-4] + "FH" + ".pdb"
    p1               = subprocess.Popen(["C:\\Windows\\System32\\wsl.exe", "-d", "Ubuntu-18.04", path_reduce, "-BUILD", path_input_files2 + pdb_file, ">", path_output_files + output_filename], #whitespaces are automatically added after each comma in the list
                                        stdout = subprocess.PIPE, 
                                        stderr = subprocess.PIPE, 
                                        shell = False)
    out, err = p1.communicate()
#     print(out.decode("utf-8")) #do not print the outputs of the reduce command as they are very long & can cause the notebook to hang
#     print(err.decode("utf-8"))

## For standalone Ubuntu

In [ ]:
import os
import subprocess

path_input_files  = "/home/rsankar/prathvi/files_pdb/"
path_output_files = "/home/rsankar/prathvi/files_molprobity/"

for pdb_file in os.listdir(path_input_files):
    print("processing", pdb_file)
    output_filename  = pdb_file[0:len(pdb_file)-4] + "FH" + ".pdb"
    p1               = subprocess.Popen("reduce -BUILD " + path_input_files + pdb_file + " > " + path_output_files + output_filename, stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True) #if "shell = False" is used, I get the "file not found error"
    out, err = p1.communicate()
#     print(out.decode("utf-8")) #do not print the outputs of the reduce command as they are very long & can cause the notebook to hang
#     print(err.decode("utf-8"))

# [Manual step] Create a new tab in the excel file downloaded from PDB which contains only the unique entity IDs

# Separate the unique entity IDs in batches of 1000
You can use the same code which you used to create batches for PDB IDs

# Download XML files from PDBTM and save response codes to a JSON file
I am saving the response codes to a json file because I need to know which XML files downloaded and which didn't. Once you open this file, you can use the regex ("\w{4}")(:\s)(?!200,?)(\d+,) to find out all the PDB IDs whose response codes were not equal to 200.

In [12]:
%reset -f

In [11]:
import requests
import json
import os
import pandas as pd
import pprint

path_response_codes   = "C:\\Users\\Prathvi\\Downloads\\response codes pdbtm.json"
path_downloaded_files = "C:\\Users\\Prathvi\\Downloads\\PDBTM\\"
if not os.path.isdir(path_downloaded_files):
    os.makedirs(path_downloaded_files)

df1   = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Project 03\\files_excel\\all mpstruc entries taken from pdb via advanced search.xlsx", sheet_name = "Unique PDB IDs")
dict1 = dict()
for i in df1.itertuples():
    row      = list(i)
    pdb_id   = row[1].lower()
    midword  = pdb_id[1:3]
    print("processing:", pdb_id.upper())
    # url      = "http://pdbtm.enzim.hu/data/database/" + midword + "/" + pdb_id + ".xml" # this was the old url
    url      = "https://pdbtm.unitmp.org/api/v1/entry/" + pdb_id + ".xml"                 # this is the new url
    response = requests.get(url, allow_redirects = True)
    status   = response.status_code
    dict1.setdefault(pdb_id, status)
    if status == 200:
        with open(path_downloaded_files + pdb_id + ".xml", "wb") as file1:
            file1.write(response.content)

with open(path_response_codes, "w") as file2:
    json.dump(dict1, file2, sort_keys = True, indent = 0)

for key, value in dict1.items():
    if value != 200:
        print(key)

processing: 1A0S
processing: 1A0T
processing: 1AF6
processing: 1AQT
processing: 1BH3
processing: 1BL8
processing: 1BMF
processing: 1BT9
processing: 1BY3
processing: 1BY5
processing: 1C8R
processing: 1C8S
processing: 1COW
processing: 1E12
processing: 1E54
processing: 1EFR
processing: 1EHK
processing: 1EK9
processing: 1F02
processing: 1FCP
processing: 1FEP
processing: 1FI1
processing: 1FW2
processing: 1FW3
processing: 1FX8
processing: 1GFM
processing: 1GFN
processing: 1GFO
processing: 1GFP
processing: 1GFQ
processing: 1H2S
processing: 1H68
processing: 1H6S
processing: 1HXT
processing: 1HXU
processing: 1HXX
processing: 1HYN
processing: 1I78
processing: 1ILD
processing: 1ILZ
processing: 1IM0
processing: 1IWO
processing: 1J4N
processing: 1JB0
processing: 1K4C
processing: 1K4D
processing: 1KB9
processing: 1KF6
processing: 1KFY
processing: 1KG8
processing: 1KG9
processing: 1KGB
processing: 1KMO
processing: 1KMP
processing: 1KPK
processing: 1KPL
processing: 1KQF
processing: 1KQG
processing: 1L

In [10]:
%reset -f

# Extract TM info from PDBTM XML files and save in a excel file

PDBTM XML sample schema (see the "Region/" tag): http://pdbtm.enzim.hu/data/pdbtm.xsd

In [15]:
%reset -f

In [18]:
import xml.etree.ElementTree as ET
import os
import pprint
import pandas as pd
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import range_to_table
import importlib
importlib.reload(range_to_table)

xml_files = "D:\\My Data\\Study Stuff\\PhD Work\\Project 03\\files_tm_info\\PDBTM\\xml files_V2\\"
dict1     = dict()

for file in os.listdir(xml_files):
    pdb_id                  = file[0:len(file) - 4].upper()
    tree                    = ET.parse(xml_files + file)
    root                    = tree.getroot()
    segments                = list()
    chains                  = list()
    chains_with_num_tm      = list()
    dict_all_segments_spans = dict()
    print("processing:", pdb_id)
    for t in root.iter():

        if t.tag == "{https://pdbtm.unitmp.org}TMTYPE": # as of 20240916, the xml file format has been slightly updayes due to which the link in the "if" statement has been updated {http://pdbtm.enzim.hu}
            protein_type = t.text

        if t.tag == "{https://pdbtm.unitmp.org}CHAIN": # {http://pdbtm.enzim.hu}
            chain_id             = t.attrib["CHAINID"]
            chain_id_with_num_tm = t.attrib["NUM_TM"]
            chains.append(chain_id)
            chains_with_num_tm.append(chain_id + "(" + chain_id_with_num_tm + ")")
            for child in t:
                if child.tag == "{https://pdbtm.unitmp.org}REGION": # {http://pdbtm.enzim.hu}
                    pdb_beg      = child.attrib["pdb_beg"]
                    pdb_end      = child.attrib["pdb_end"]
                    segment_type = child.attrib["type"]
                    dict_all_segments_spans.setdefault(chain_id, []).append(pdb_beg + "-" + pdb_end + "(" + segment_type + ")" + ",")
            dict_all_segments_spans[chain_id][-1] = dict_all_segments_spans[chain_id][-1][0:-1] + "." #this will replace the last occurence of "," with "." from the last element of the list such that it changes from "10-20(H)," to "10-20(H)."

    string_chains             = ",".join(chains)
    string_chains_with_num_tm = ",".join(chains_with_num_tm)
    string_all_segments_spans = str()

    for key, value in dict_all_segments_spans.items():
        string_all_segments_spans = string_all_segments_spans + key + ":" + "".join(value)

    dict1.setdefault("PDB ID",                                       []).append(pdb_id)
    dict1.setdefault("Protein type",                                 []).append(protein_type)
    dict1.setdefault("Total number of chains",                       []).append(string_chains)
    dict1.setdefault("TM segments per chain",                        []).append(string_chains_with_num_tm)
    dict1.setdefault("All segments per chain with segment notation", []).append(string_all_segments_spans)

# # pprint.pprint(dict1)
df1 = pd.DataFrame(dict1)
workbook_path = "D:\\My Data\\Study Stuff\\PhD Work\\Project 03\\files_excel\\PDBTM segments info V2.xlsx"
df1.to_excel(workbook_path, index = False, na_rep = "NA")
range_to_table.range_to_table(workbook_path)

processing: 1A0S
processing: 1A0T
processing: 1AF6
processing: 1AQT
processing: 1BH3
processing: 1BL8
processing: 1BMF
processing: 1BT9
processing: 1BY3
processing: 1BY5
processing: 1C8R
processing: 1C8S
processing: 1COW
processing: 1E12
processing: 1E54
processing: 1EFR
processing: 1EHK
processing: 1EK9
processing: 1F02
processing: 1FCP
processing: 1FEP
processing: 1FI1
processing: 1FW2
processing: 1FW3
processing: 1FX8
processing: 1GFM
processing: 1GFN
processing: 1GFO
processing: 1GFP
processing: 1GFQ
processing: 1H2S
processing: 1H68
processing: 1H6S
processing: 1HXT
processing: 1HXU
processing: 1HXX
processing: 1HYN
processing: 1I78
processing: 1ILD
processing: 1ILZ
processing: 1IM0
processing: 1IWO
processing: 1J4N
processing: 1JB0
processing: 1K4C
processing: 1K4D
processing: 1KB9
processing: 1KF6
processing: 1KFY
processing: 1KG8
processing: 1KG9
processing: 1KGB
processing: 1KMO
processing: 1KMP
processing: 1KPK
processing: 1KPL
processing: 1KQF
processing: 1KQG
processing: 1L

In [17]:
%reset -f

# [MANUAL STEP] Add a new sheet in the RCSB excel file which contains those PDB IDs whose XML files could not be found in PDBTM
I used excel's conditional formatting to do so. Alternatively, you can open the text file which contained the response codes of the PDBTM XML files and grab those IDs whose response code !=200

# Download JSON files of those PDB IDs from OPM whose info couldn't be found on PDBTM and save response codes to a JSON file

In [81]:
%reset -f

In [82]:
import json
import requests
import pprint
import os
import pandas as pd

url                   = "https://lomize-group-opm.herokuapp.com/primary_structures/pdbid/" 
response_codes        = dict()
path_downloaded_files = "D:\\OPM JSON files\\"
if not os.path.isdir(path_downloaded_files):
    os.makedirs(path_downloaded_files)

df1 = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\all mpstruc entries taken from pdb via advanced search.xlsx", sheet_name = "info not found in pdbtm")
for i in df1.itertuples():
    row          = list(i)
    pdb_id       = row[1].lower() #OPM API throws error if the PDB ID is provided in uppercase
    download_url = url + pdb_id
    response     = requests.get(download_url, allow_redirects = True)
    print(pdb_id, "processed with return code:", response.status_code)
    response_codes.setdefault(pdb_id, response.status_code)

    if response.status_code == 200:
        json_data = json.loads(str(response.content, encoding='utf-8'))
        with open(path_downloaded_files + pdb_id + ".json", "w") as file1:
            json.dump(json_data, file1, indent = 0)

with open("D:\\response codes opm.json", "w") as file2:
    json.dump(response_codes, file2, sort_keys = True, indent = 0)

4v8k processed with return code: 500
4yuu processed with return code: 200
6uwi processed with return code: 500
6w1o processed with return code: 500
6w1p processed with return code: 500
6w1q processed with return code: 500
6w1r processed with return code: 500
6w1t processed with return code: 500
6w1u processed with return code: 500
6w1v processed with return code: 500
6yvr processed with return code: 200
6yy5 processed with return code: 500
6z2n processed with return code: 200
6z33 processed with return code: 500
6z4q processed with return code: 200
6z4v processed with return code: 200
6z8n processed with return code: 200
6za8 processed with return code: 200
6zin processed with return code: 200
6zz5 processed with return code: 500
6zzh processed with return code: 500
7a0g processed with return code: 500
7obw processed with return code: 200
7pj5 processed with return code: 500
7pp1 processed with return code: 200
7q7p processed with return code: 500
7q7q processed with return code: 500
7

# [MANUAL STEP] Manually download the OPM JSON files of those PDB IDs which failed to download using code
Why code failed to download them? It is because the info of these IDs was either not available altogether on OPM or they were registered with an alternate PDB ID which represented the same structure.

# [MANUAL STEP] Add a new sheet in the RCSB excel file which contains those PDB IDs whose JSON files could neither be found on PDBTM nor OPM

# Download XML files of those PDB IDs from UNIPROT whose info could neither be found on PDBTM nor OPM and save response codes to a JSON file

In [2]:
%reset -f

In [3]:
import json
import requests
import os
import pandas as pd

url1                   = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/" #URL to grab uniprot accession IDs corresponding to PDB IDs
url2                   = "https://rest.uniprot.org/uniprotkb/"              #URL to grab xml files corresponding to uniprot accession IDs
response_codes1        = dict()                                             #dictionary to save the response codes generated while searching for uniprot accession IDs corresponding to PDB IDs
response_codes2        = dict()                                             #dictionary to save the response codes generated while searching for xml files corresponding to uniprot accession IDs
path_downloaded_files  = "D:\\UNIPROT XML files\\"
if not os.path.isdir(path_downloaded_files):
    os.makedirs(path_downloaded_files)

df1 = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\all mpstruc entries taken from pdb via advanced search.xlsx", sheet_name = "info not found in opm or pdbtm")
for i in df1.itertuples():
    row           = list(i)
    pdb_id        = row[1].lower()
    download_url1 = url1 + pdb_id
    response1     = requests.get(download_url1, allow_redirects = True)
    print("response code for the request to find a uniprot accession ID corresponding to the pdb id:", pdb_id, "is", response1.status_code)
    response_codes1.setdefault(pdb_id, response1.status_code)

    if response1.status_code == 200:
        json_data            = json.loads(str(response1.content, encoding='utf-8'))
        uniprot_accession_id = list(json_data[pdb_id]["UniProt"].keys())[0]
        download_url2        = url2 + uniprot_accession_id + ".xml"
        response2            = requests.get(download_url2, allow_redirects = True)
        print("response code for downloading the xml file for uniprot acccession ID:", uniprot_accession_id, "is", response2.status_code)
        response_codes2.setdefault(uniprot_accession_id, response2.status_code)

        if response2.status_code == 200:
            with open(path_downloaded_files + pdb_id + ".xml", "wb") as file1:
                file1.write(response2.content)

with open("D:\\response codes for searching uniprot accession IDs corresponding to PDB IDs.json", "w") as file2:
    json.dump(response_codes1, file2, sort_keys = True, indent = 0)

with open("D:\\response codes uniprot.json", "w") as file3:
    json.dump(response_codes2, file3, sort_keys = True, indent = 0)    

response code for the request to find a uniprot accession ID corresponding to the pdb id: 6uwi is 200
response code for downloading the xml file for uniprot acccession ID: O32739 is 200
response code for the request to find a uniprot accession ID corresponding to the pdb id: 6zz5 is 200
response code for downloading the xml file for uniprot acccession ID: A0A1Q4NVM7 is 200
response code for the request to find a uniprot accession ID corresponding to the pdb id: 6zzh is 200
response code for downloading the xml file for uniprot acccession ID: A0A1Q4NVM7 is 200
response code for the request to find a uniprot accession ID corresponding to the pdb id: 7a0g is 200
response code for downloading the xml file for uniprot acccession ID: A0A1Q4NVM7 is 200
response code for the request to find a uniprot accession ID corresponding to the pdb id: 7pj5 is 200
response code for downloading the xml file for uniprot acccession ID: P59206 is 200
response code for the request to find a uniprot accession 

# Find residuewise count of all residues <font color="red">in non-redundant chains</font> of the dataset with <font color="red">TM segment info for each residue</font>

The V1 of this code is similar to the one in the self contacts jupyter notebook. The only difference is that it has been slightly modified to also provide information about the location of the residue in the transmembrane region

## V1

In [7]:
%reset -f

In [ ]:
import re
import os
import pprint
import pandas as pd
from Bio.PDB import *
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import range_to_table
import importlib
importlib.reload(range_to_table)

dict_segment_meanings = {"1" : "Extracellular side", 
                        "2" : "Intracellular/Periplasmic side", 
                        "H" : "Transmembrane alpha helix", 
                        "B" : "Transmembrane beta strand", 
                        "C" : "Coiled Structure", 
                        "I" : "Membrane embedded region not interacting with lipids", 
                        "L" : "Membrane embedded region not tranversing the membrane (loop)", 
                        "U" : "Unknown/missing residue", 
                        "F" : "Interfacial helix"}

dict1    = {'ALA': "A", 
            'ARG': "R", 
            'ASN': "N", 
            'ASP': "D", 
            'CYS': "C", 
            'GLN': "Q", 
            'GLU': "E", 
            'GLY': "G", 
            'HIS': "H", 
            'ILE': "I", 
            'LEU': "L", 
            'LYS': "K", 
            'MET': "M", 
            'PHE': "F", 
            'PRO': "P", 
            'SER': "S", 
            'THR': "T", 
            'TRP': "W", 
            'TYR': "Y", 
            'VAL': "V"}

dict_resnature = {("GLY", "ALA", "VAL", "ILE", "LEU", "MET", "PRO") : "Hydrophobic", 
                ("PHE", "TYR", "TRP") : "Aromatic", 
                ("LYS", "ARG", "HIS") : "Polar charged positive", 
                ("ASP", "GLU") : "Polar charged negative", 
                ("CYS", "SER", "THR", "ASN", "GLN") : "Polar uncharged"}

dictdata  = dict()
pdbparser = PDBParser(PERMISSIVE = True, QUIET = True)
#^################################################################################################
#^ Create a dictionary containing PDB IDs and their cooresponding unique chains:
#^################################################################################################
molprobity_files = "D:\\PhD Work\\Project 03\\files_pdb\\all molprobity files\\CIF+PDB\\"
df_pdbtm         = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\SHBs\\PDBTM segments info.xlsx") # this dataframe stores the info on transmembrane segments
with open ("D:\\PhD Work\\Project 03\\files_pisces\\results_7\\PISCES_results.txt") as file1:
    list1 = file1.readlines()
    list1 = sorted(list1)
    file1.close()
    dict_id_chains = dict()
    for i in list1:
        dict_id_chains.setdefault(i[0:4], []).append(i[4]) #key will be PDB ID & value will be the no. of distinct chains in that PDB ID
    # print(len(dict_id_chains.keys()))
    # print(dict_id_chains.keys())

#^############################################################################
#^ Iterate over each non-redundant chain of each PDB ID in the dictionary:
#^############################################################################
for pdbid, distinct_chains in dict_id_chains.items():
    pdb_id_molprobity = next((molprobity_file_name for molprobity_file_name in os.listdir(molprobity_files) if molprobity_file_name[0:4].upper() == pdbid), None) # "pdb_id_molprobity" will also have extension of the file so that pdbparser can read it from disk
    if pdb_id_molprobity != None:
        print("processing:", pdb_id_molprobity)

        #^##################################################################################################
        #^ Grab the TM annotations of all the distinct chains of the PDB ID:
        #^##################################################################################################
        df_temp     = df_pdbtm[(df_pdbtm["PDB ID"]) == pdbid]
        if len(df_temp) == 0: # it will be true when the PDB ID in the iteration has no corresponding entry in the dataframe e.g., for the PDB IDs "7PJ5", "6ZZ5", "7RF1", "4YUU"
            print("PDB ID not found on PDBTM")
            tm_segments2 = "PDB ID not found on PDBTM" # it will stay empty in this case
        if len(df_temp) != 0:
            row         = df_temp.iloc[0].tolist()     # grab the 1st row of the dataframe (in theory the dataframe should have only one row but just to be safe) & convert to list
            tm_segments = row[4]                       # grab the 5th element of the list as it contains info on tm segments
            if type(tm_segments) == float:             # it will be true when the row corresponding to the PDB ID in the iteration does exist in the excel file but no TM info is available for it (if you print this info, it will be nan)
                print("PDB ID found on PDBTM but TM info not found")
                tm_segments2 = "PDB ID found on PDBTM but TM info not found" # it will stay empty in this case
            if type(tm_segments) != float:            # it will be true only when the TM info is available
                tm_segments  = tm_segments.split(".") # if the PDB file had two chains A & B, then this list will have 3 elements & will look like ["A:71-76(2),77-85(B),86-104(1),105-116(I)", "B:71-76(2),77-85(B),86-104(1)", ""]
                tm_segments  = tm_segments[0:-1]      # the last element of the list is empty so I am removing it
                tm_segments2 = dict()                 # it will contain TM segment info of all the distinct chains of the PDB file in the current iteration; see the last line of this section to know how this dict will look like
                # print(tm_segments)
                for tm_segment in tm_segments:
                    match1 = re.search(r"(\w)(:)(.+)", tm_segment) # it will recognize text like: "A:71-76(2),77-85(B),86-104(1),105-116(I)"
                    if match1:
                        if match1.group(1) in distinct_chains:
                            tm_segments2.setdefault(match1.group(1), {}) # it will look like: {"A" : {}} where "A" is the chain ID
                            list_segments = match1.group(3).split(",")   # it will look like: [71-76(2), 77-85(B), 86-104(1), 105-116(I)]
                            for subsegment in list_segments:
                                match2 = re.search(r"(\d+)(-)(\d+)(\()(\w)(\))", subsegment) # it will recognize text like: "87-104(H)" and "122-130(2)"
                                if match2:
                                    segment_type  = match2.group(5)
                                    segment_start = int(match2.group(1)) 
                                    segment_end   = int(match2.group(3))
                                    tm_segments2[match1.group(1)].setdefault(segment_type, []).append([segment_start, segment_end]) # it will look like: {'A': {'H': [[33, 37], [46, 51]], 'B': [[33, 37], [46, 51]]}}

        #^##################################################################################################
        #^ Grab the info regarding each residue in a dictionary:
        #^##################################################################################################
        structure = pdbparser.get_structure("protein", molprobity_files + pdb_id_molprobity)
        residues  = structure.get_residues()
        for residue in residues:
            if residue.get_resname() in dict1.keys() and residue.get_full_id()[2] in distinct_chains and residue.get_full_id()[3][0] == " " and residue.get_full_id()[3][2] == " ": #if the residue belongs to a unique chain & is a non-heteroatom entity & has no insertion code
                resno    = residue.get_full_id()[3][1]
                reschain = residue.get_full_id()[2]
                dictdata.setdefault("PDB ID",                 []).append(pdb_id_molprobity[0:4].upper())
                dictdata.setdefault("Chain ID",               []).append(reschain)
                dictdata.setdefault("PDB ID/Chain ID",        []).append(pdb_id_molprobity[0:4].upper() + "/" + residue.get_full_id()[2])
                dictdata.setdefault("Resname (3 letters)",    []).append(residue.get_resname())
                dictdata.setdefault("Resname (1 letter)",     []).append(dict1[residue.get_resname()])
                dictdata.setdefault("Resno",                  []).append(resno)
                for key, value in dict_resnature.items():
                    if residue.get_resname() in key:
                        dictdata.setdefault("Residue nature", []).append(value)

                #^##################################################################################################
                #^ Assign TM segment info to the residue in the current iteration:
                #^##################################################################################################
                if type(tm_segments2) == str:
                    tm_segment_info = "AAAA_" + tm_segments2
                if type(tm_segments2) == dict:
                    if reschain not in list(tm_segments2.keys()): # sometimes the dict "tm_segments2" contains TM segment info corresponding to the auth asym IDs instead of the biopython chain IDs. In such cases, this condition will become true
                        flag_donor      = True
                        tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to donor chain not found"
                    if reschain in list(tm_segments2.keys()): 
                        flag_donor = True # it will become False only when the residue's TM segment info is found
                        for segment_type, segment_ranges in tm_segments2[reschain].items():
                            for segment_range in segment_ranges:
                                segment_range_start = segment_range[0]
                                segment_range_end   = segment_range[1]
                                if resno in range(segment_range_start, segment_range_end + 1):
                                    tm_segment_info = dict_segment_meanings[segment_type]
                                    flag_donor      = False
                        if flag_donor == True:
                            tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to donor residue not found"
                dictdata.setdefault("TM segment info", []).append(tm_segment_info)

df1 = pd.DataFrame(dictdata)
df1.sort_values("PDB ID", inplace = True)
workbook_path = "D:\\Resinfo of residues in non-redundant chains in TM dataset.xlsx"
df1.to_excel(workbook_path, index = False, na_rep = "NA")
range_to_table.range_to_table(workbook_path)

processing: 1kqfFH.pdb
processing: 1nkzFH.pdb
processing: 1qjpFH.pdb
processing: 1u7gFH.pdb
processing: 1y5iFH.pdb
processing: 1yc9FH.pdb
processing: 2a65FH.pdb
processing: 2ervFH.pdb
processing: 2gr8FH.pdb
processing: 2gufFH.pdb
processing: 2h88FH.pdb
processing: 2jdiFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2ns1FH.pdb
processing: 2onsFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2uuiFH.pdb
processing: 2vqgFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2wjrFH.pdb
processing: 2x55FH.pdb
processing: 2xtvFH.pdb
processing: 2yh5FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2yh9FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2yhcFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 3aehFH.pdb
processing: 3b9wFH.pdb
processing: 3ck6FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 3fidFH.pdb
processing: 3gp6FH.pdb
processing: 3ldcFH.pdb
processing: 3m73FH.pdb
processing: 3ml3FH.pdb
PD

In [8]:
%reset -f

## Some old code

In [8]:
import pandas as pd
import re
import os
import pprint
import json
from Bio import SeqIO
from Bio.PDB import *
import importlib
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import residue_counter
importlib.reload(residue_counter)

pdbparser              = PDBParser(PERMISSIVE = True, QUIET = True)
fasta_cdhit            = SeqIO.parse("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_fasta\\cdhit results\\cdhit sequences whose segment info is found on pdbtm.fasta", "fasta")
molprobity_files       = "D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_pdb\\all molprobity files\\CIF+PDB\\"
df_rcsb                = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\all mpstruc entries taken from pdb via advanced search.xlsx", sheet_name = "EntID with chain ID")
df_pdbtm               = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Part 3\\files_excel\\segments info taken from pdbtm xml files.xlsx")
relevant_segments      = ["(H)", "(B)", "(C)", "(I)", "(L)"]
list_relevant_residues = ["MET", "CYS", "SER", "THR"]
dictdata         = {'ALA': {}, 'ARG': {}, 'ASN': {}, 'ASP': {}, 'CYS': {}, 'GLN': {}, 'GLU': {}, 'GLY': {}, 'HIS': {}, 'ILE': {}, 'LEU': {}, 'LYS': {}, 'MET': {}, 'PHE': {}, 'PRO': {}, 'SER': {}, 'THR': {}, 'TRP': {}, 'TYR': {}, 'VAL': {}}

for entry in fasta_cdhit:
    match1 = re.search(r"(\w{4})(_)(\d+)(\|)(.+)", entry.id)                                                                     #"entry.id" looks like "1AF6_1|Chains A, B, C|MALTOPORIN|Escherichia coli (562)"
    if match1:
        #grab the pdb id and entity id from cdhit sequence
        pdb_id_cdhit                = match1.group(1).upper()
        pdb_id_with_entity_id_cdhit = match1.group(1) + match1.group(2) + match1.group(3)
        #grab the cdhit entity id's corresponding "auth asym id" from rcsb excel file
        chain_auth_asym_id          = df_rcsb[(df_rcsb["Entity ID"] == pdb_id_with_entity_id_cdhit)].iloc[0].tolist()[2]         #iloc[0].tolist() will contain the very first row (as a list) whose entity id matches the cdhit entity id. The "[2]" denotes its 3rd element which is the "auth asym id"
        #grab the tm segments of all the chains
        segments_raw                = list()
        segments_raw                = df_pdbtm[(df_pdbtm["PDB ID"]) == pdb_id_cdhit].iloc[0].tolist()[4]
        segments_raw                = segments_raw.split(".")                                                                    # it will look like ["P:71-76(2),77-85(B),86-104(1),105-116(I)", "Q:71-76(2),77-85(B),86-104(1)", ""]
        segments_relevant_chain     = dict()
        #grab the tm segments of the chain whose auth asym id is extracted
        for j in segments_raw:
            match1 = re.search(r"(\w)(:)(.+)", j)
            if match1 and match1.group(1) == chain_auth_asym_id:
                list1 = match1.group(3).split(",")                                                                               #it will look like ["71-76(2)","77-85(B)","86-104(1)","105-116(I)"]
                for k in list1:
                    match2 = re.search(r"(.+)(\(\w\))", k)
                    if match2 and match2.group(2) in relevant_segments:
                        segments_relevant_chain.setdefault("HBCIL", []).append(match2.group(1))
                    if match2 and match2.group(2) not in relevant_segments:
                        segments_relevant_chain.setdefault("12U", []).append(match2.group(1))                                    #it will look like {'12U': ['71-76', '86-104'], 'HBCIL': ['77-85', '105-116']}
#         print(pdb_id_cdhit,chain_auth_asym_id)
#         pprint.pprint(segments_relevant_chain)

        #open the corresponding pdb file and grab all the donor atoms in all the tm segments of the relevant chain
        for molprobity_file_name in os.listdir(molprobity_files):
            pdb_id_molprobity = molprobity_file_name
            if pdb_id_cdhit == pdb_id_molprobity[0:4].upper():
                print("processing:", pdb_id_molprobity)

                #create a list containing info about disulfide bonded cysteines in the current pdb file
                with open(molprobity_files + pdb_id_molprobity) as file2:
                    list_ssbond = file2.readlines()
                    file2.close()
                    list_disulfides = list()
                    for line_ssbond in list_ssbond:
                        match_ssbond = re.search(r"(SSBOND)(\s\*{3}\s)(CYS|CSX|CSO|DCY|DTT)(\s)(\w+)(\s+)(\d+)(\s+)(CYS|CSX|CSO|DCY|DTT)(\s)(\w+)(\s+)(\d+)(.+)", line_ssbond) #this regex is written and tested to be valid for all "SSBOND" lines across the globular and tm dataset
                        if match_ssbond:
                            if "CYS" in match_ssbond.group(3): # I have used this condition because I have found cases where there is a disulfide bond between non-CYS entities or between CYS and a non-CYS entity
                                list_disulfides.append(str(match_ssbond.group(5)) + "." + str(match_ssbond.group(7))) # "reschain" plus "resno"
                            if "CYS" in match_ssbond.group(9):
                                list_disulfides.append(str(match_ssbond.group(11)) + "." + str(match_ssbond.group(13))) # "reschain" plus "resno"
                # pprint.pprint(list_disulfides)

                structure = pdbparser.get_structure("protein", molprobity_files + molprobity_file_name)
                residue_counter.rescounter_tm(structure, chain_auth_asym_id, segments_relevant_chain, pdb_id_with_entity_id_cdhit, pdb_id_molprobity, dictdata, list_disulfides)

with pd.ExcelWriter("D:\\Residuewise counts in TM segments of unique chains in tm dataset.xlsx", engine = "xlsxwriter") as writer:
    dict_summary = dict()
    for key, value in dictdata.items():
        df1 = pd.DataFrame(value)
        dict_summary.setdefault("Residue", []).append("Total count of " + key)
        dict_summary.setdefault("Count",   []).append(len(df1))
        df1.sort_values("PDB ID", inplace = True)
        df1.to_excel(writer, sheet_name = key, index = False, na_rep = "NA")

    df_summary = pd.DataFrame(dict_summary)
    df_summary.to_excel(writer, sheet_name = "Summary", index = False, na_rep = "NA")

processing: 1af6FH.pdb
processing: 1e54FH.pdb
processing: 1gfmFH.pdb
processing: 1h6sFH.pdb
processing: 1ildFH.pdb
processing: 1kmoFH.pdb
processing: 1nqeFH.pdb
processing: 1p4tFH.pdb
processing: 1phoFH.pdb
processing: 1qffFH.pdb
processing: 1qjpFH.pdb
processing: 1t16FH.pdb
processing: 1xkwFH.pdb
processing: 2ervFH.pdb
processing: 2gr7FH.pdb
processing: 2grxFH.pdb
processing: 2hdiFH.pdb
processing: 2iwvFH.pdb
processing: 2o4vFH.pdb
processing: 2vqiFH.pdb
processing: 2x4mFH.pdb
processing: 3a2sFH.pdb
processing: 3anzFH.pdb
processing: 3bryFH.pdb
processing: 3bs0FH.pdb
processing: 3cslFH.pdb
processing: 3dzmFH.pdb
processing: 3efmFH.pdb
processing: 3jtyFH.pdb
processing: 3sloFH.pdb
processing: 3sy9FH.pdb
processing: 3sybFH.pdb
processing: 3szdFH.pdb
processing: 3szvFH.pdb
processing: 3t0sFH.pdb
processing: 3v89FH.pdb
processing: 4b7oFH.pdb
processing: 4c00FH.pdb
processing: 4c69FH.pdb
processing: 4d5dFH.pdb
processing: 4e1sFH.pdb
processing: 4epaFH.pdb
processing: 4pr7FH.pdb
processing:

# Use stride to generate files containing the <font color="red">secondary structure</font> info for molprobity files

Stride doesn't work for CIF files.

Stride also generates phi psi values which match the phi psi values generated by chimeraX.

In [ ]:
%reset -f

In [ ]:
import os
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import stride_sec_str_and_phi_psi_files_generator
import importlib
importlib.reload(stride_sec_str_and_phi_psi_files_generator)

path_stride_files1 = "D:\\files_stride_tm\\"
path_stride_files2 = "/mnt/d/files_stride_tm/"
if not os.path.isdir(path_stride_files1):
    os.makedirs(path_stride_files1)

path_molprobity_files  = "D:\\files_molprobity\\tm\\"
path_molprobity_files2 = "/mnt/d/files_molprobity/tm/" # see the function to know why I passed the molprobity files path twice
stride_sec_str_and_phi_psi_files_generator.file_generator(path_molprobity_files, path_molprobity_files2, path_stride_files2, path_stride_files1)

processing 1a0sFH.pdb
processing 1a0tFH.pdb
processing 1af6FH.pdb
processing 1aqtFH.pdb
processing 1bh3FH.pdb
processing 1bl8FH.pdb
processing 1bmfFH.pdb
processing 1bt9FH.pdb
processing 1by3FH.pdb
processing 1by5FH.pdb
processing 1c8rFH.pdb
processing 1c8sFH.pdb
processing 1cowFH.pdb
processing 1e12FH.pdb
processing 1e54FH.pdb
processing 1efrFH.pdb
processing 1ehkFH.pdb
processing 1ek9FH.pdb
processing 1f02FH.pdb
processing 1fcpFH.pdb
processing 1fepFH.pdb
processing 1fi1FH.pdb
processing 1fw2FH.pdb
processing 1fw3FH.pdb
processing 1fx8FH.pdb
processing 1gfmFH.pdb
processing 1gfnFH.pdb
processing 1gfoFH.pdb
processing 1gfpFH.pdb
processing 1gfqFH.pdb
processing 1h2sFH.pdb
processing 1h68FH.pdb
processing 1h6sFH.pdb
processing 1hxtFH.pdb
processing 1hxuFH.pdb
processing 1hxxFH.pdb
processing 1hynFH.pdb
processing 1i78FH.pdb
processing 1ildFH.pdb
processing 1ilzFH.pdb
processing 1im0FH.pdb
processing 1iwoFH.pdb
processing 1j4nFH.pdb
processing 1jb0FH.pdb
processing 1k4cFH.pdb
processing

# Use Naccess to calculate <font color="red">SASA (in Åsquare)</font> for all molprobity files

It generates 3 files per PDB file

In [24]:
%reset -f

In [ ]:
import os
import subprocess

path_molprobity_files  = "D:\\files_molprobity\\"
path_molprobity_files2 = "/mnt/d/files_molprobity/"

for pdb_file in os.listdir(path_molprobity_files):
    print("processing", pdb_file)
    p1 = subprocess.Popen(["C:\\Windows\\System32\\wsl.exe", "/home/prathvi/Naccess/naccess", path_molprobity_files2 + pdb_file], #whitespaces are automatically added after each comma in the list
                          stdout = subprocess.PIPE, 
                          stderr = subprocess.PIPE, 
                          shell = False) #the output files are created in the directory where the jupyter notebook is located and not where the source pdb files are located
    out, err = p1.communicate()
    print(out.decode("utf-8"))
    print(err.decode("utf-8"))


# Use chimeraX to generate files containing <font color="red">phi psi chi1</font> values of all molprobity files

In [ ]:
#NOTE: The cell in which you execute this code will continue to run until you close chimera's GUI window.
import subprocess

p1 = subprocess.Popen(["C:\\Program Files\\ChimeraX\\bin\\ChimeraX.exe", "--script", "D:\\phi_psi_chi1_files_generator_chimeraX.py"], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
out, err = p1.communicate()
print(out.decode("utf-8"))
print(err.decode("utf-8"))

# Save the <font color="red">resolution</font> of molprobity files to a JSON dictionary

In [22]:
%reset -f

In [23]:
import os
import re
import json

dict_resolutions = dict()
dict_path_regex  = {
                    "D:\\files_molprobity\\tm\\"               : r"(REMARK   2 RESOLUTION.)(\s+)(\d\.\d+)(\s+ANGSTROMS.)", #the "hybrid36" version of CIF molprobity files dont have the resolution info so I had to extract resolution separately from their original CIF files
                    "D:\\files_molprobity\\CIF original tm\\"  : r"(_reflns.d_resolution_high)(\s+)(\d\.\d+)"
                    }

for path_molprobity_files, regex in dict_path_regex.items():
    for pdb_file in os.listdir(path_molprobity_files):
        with open(path_molprobity_files + pdb_file) as file1:
            print("processing:", pdb_file)
            list1 = file1.readlines()
            file1.close()
            for line in list1:
                match1 = re.search(regex, line)
                if match1:
                    dict_resolutions.setdefault(pdb_file[0:4].upper(), float(match1.group(3))) #for both "pdb" and "cif" files, "match1.group(3)" contains the resolution

dict_resolutions = dict(sorted(dict_resolutions.items()))
with open("D:\\molprobity files resolutions tm.json", 'w') as file1:
    json.dump(dict_resolutions, file1, indent = 1)

#######################################################################################################
# Open the created dictionary
#######################################################################################################
# import json
# with open('D:\\molprobity files resolutions tm.json', 'r') as file1:
#     dict1 = json.load(file1)
# print(len(dict1))

processing: 1a0sFH.pdb
processing: 1a0tFH.pdb
processing: 1af6FH.pdb
processing: 1aqtFH.pdb
processing: 1bh3FH.pdb
processing: 1bl8FH.pdb
processing: 1bmfFH.pdb
processing: 1bt9FH.pdb
processing: 1by3FH.pdb
processing: 1by5FH.pdb
processing: 1c8rFH.pdb
processing: 1c8sFH.pdb
processing: 1cowFH.pdb
processing: 1e12FH.pdb
processing: 1e54FH.pdb
processing: 1efrFH.pdb
processing: 1ehkFH.pdb
processing: 1ek9FH.pdb
processing: 1f02FH.pdb
processing: 1fcpFH.pdb
processing: 1fepFH.pdb
processing: 1fi1FH.pdb
processing: 1fw2FH.pdb
processing: 1fw3FH.pdb
processing: 1fx8FH.pdb
processing: 1gfmFH.pdb
processing: 1gfnFH.pdb
processing: 1gfoFH.pdb
processing: 1gfpFH.pdb
processing: 1gfqFH.pdb
processing: 1h2sFH.pdb
processing: 1h68FH.pdb
processing: 1h6sFH.pdb
processing: 1hxtFH.pdb
processing: 1hxuFH.pdb
processing: 1hxxFH.pdb
processing: 1hynFH.pdb
processing: 1i78FH.pdb
processing: 1ildFH.pdb
processing: 1ilzFH.pdb
processing: 1im0FH.pdb
processing: 1iwoFH.pdb
processing: 1j4nFH.pdb
processing:

# <font color="#C5E90B">Find short hydrogen bonds in TM dataset (took 239 mins or ~4 hours)</font>

This code takes the non-redundant chains info from PISCES results.<br>
This code <font color="red">DOES NOT</font> require "h_bonds_cys_ser_thr_as_donor.py" or "post_processing.py" because the relevant parts of those codes have already been incorporated in this code.<br>
This code can also be used to identify normal hydrogen bonds in TM dataset BUT it can only find them on the basis of DA, HA and DHA angles parameters and NOT the HAA` parameter. Why so? Because sir asked me to find normal H-bonds in TM dataset, based on parameters given on page 8 of Saraswathi mam's paper.

In [21]:
%reset -f

In [ ]:
import re
import os
import pprint
import json
import math
import pandas as pd
import numpy as np
from Bio.PDB import *
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import post_processing
import ssec_strc_phi_psi_chi1_sasa_value_returner
import range_to_table
import importlib
importlib.reload(post_processing)
importlib.reload(ssec_strc_phi_psi_chi1_sasa_value_returner)
importlib.reload(range_to_table)

#^###############################################################################################################################################################################################
#^ Function to check whether the DA distance is less than the distance between D and any H-atom bonded to the acceptor atom or not:
#^###############################################################################################################################################################################################
def min_dist_checker(dict_donor_and_hydrogen_atoms, desired_donor_atom, desired_acceptor_atom):
	if desired_acceptor_atom.get_name() not in list(dict_donor_and_hydrogen_atoms.keys()): # if the acceptor atom is an O/N/S atom but is not part of the list "dict_donor_and_hydrogen_atoms_ON.keys()", it means that it isnt bonded to a H-atom
		return True
	if desired_acceptor_atom.get_name() in list(dict_donor_and_hydrogen_atoms.keys()):     # if the acceptor atom is bound to a H-atom
		list_of_h_atoms = list()
		for check_atom in desired_acceptor_atom.get_parent().get_atoms():
			if check_atom.get_name() in dict_donor_and_hydrogen_atoms[desired_acceptor_atom.get_name()]: # grab all the H-atoms bonded to the acceptor atom
				list_of_h_atoms.append(check_atom)
		if len(list_of_h_atoms) == 0: # if the acceptor atom should have been bonded to H-atoms but no H-atoms were actually found bonded to the acceptor atom, then return True
			return True
		if len(list_of_h_atoms) != 0:
			# print(desired_acceptor_atom.get_name(), desired_acceptor_atom.get_parent().get_resname(), list_of_h_atoms)
			dict_check_distances = dict()
			dict_check_distances.setdefault("DA distance",  np.linalg.norm(np.array(desired_donor_atom.get_coord())  - np.array(desired_acceptor_atom.get_coord())))
			dict_check_distances.setdefault("DH distances", [np.linalg.norm(np.array(desired_donor_atom.get_coord()) - np.array(hatom.get_coord())) for hatom in list_of_h_atoms])
			if dict_check_distances["DA distance"] < min(dict_check_distances["DH distances"]): #if this condition isn't satisfied, the function will return None
				return True

#^###############################################################################################################################################################################################
#^ Function to write data to dictionary:
#^###############################################################################################################################################################################################
def calculator_and_writer(desired_donor_atom, desired_hydrogen_atom, desired_acceptor_atom, pdb_id_molprobity, dictdata, dict_resolutions, molprobity_files, tm_segments2):
    distance_primary_sequence = desired_donor_atom.get_parent().get_full_id()[3][1]     - desired_acceptor_atom.get_parent().get_full_id()[3][1]
    distance_DA               = np.linalg.norm(np.array(desired_donor_atom.get_coord()) - np.array(desired_acceptor_atom.get_coord()))
    distance_HA               = np.linalg.norm(np.array(desired_hydrogen_atom.get_coord()) - np.array(desired_acceptor_atom.get_coord()))
    angle_DHA                 = math.degrees(calc_angle(desired_donor_atom.get_vector(), desired_hydrogen_atom.get_vector(), desired_acceptor_atom.get_vector()))
    distance_DA               = round(float(distance_DA), 2) # the round() method wasn't working on "distance_DA" cuz it is a numpy value so I had to convert it to a float
    distance_HA               = round(float(distance_HA), 2)
    angle_DHA                 = round(angle_DHA, 2)
    same_chain_or_not         = desired_donor_atom.get_parent().get_full_id()[2] == desired_acceptor_atom.get_parent().get_full_id()[2]
    donor_info = pdb_id_molprobity[0:4].upper() + "_" + desired_donor_atom.get_name()    + "_" + desired_donor_atom.get_parent().get_resname()    + "_" + str(desired_donor_atom.get_parent().get_full_id()[3][1])    + "_" + str(desired_donor_atom.get_parent().get_full_id()[2]) # it will be of the format "PDBID_donatom_donresname_donresno_donreschain"
    accep_info = pdb_id_molprobity[0:4].upper() + "_" + desired_acceptor_atom.get_name() + "_" + desired_acceptor_atom.get_parent().get_resname() + "_" + str(desired_acceptor_atom.get_parent().get_full_id()[3][1]) + "_" + str(desired_acceptor_atom.get_parent().get_full_id()[2])

    #^###############################################################################
    #^ Code to assign TM segments to donor & acceptor residues:
    #^###############################################################################
    if type(tm_segments2) == str:
        donor_tm_segment_info = "AAAA_" + tm_segments2
        accep_tm_segment_info = "AAAA_" + tm_segments2
    if type(tm_segments2) == dict:
        donor_reschain = desired_donor_atom.get_parent().get_full_id()[2]
        accep_reschain = desired_acceptor_atom.get_parent().get_full_id()[2]
        donor_resno    = desired_donor_atom.get_parent().get_full_id()[3][1]
        accep_resno    = desired_acceptor_atom.get_parent().get_full_id()[3][1]
        # The following segment's code is written for two conditions:
            # 1. TM info of donor/acceptor chain not found
            # 2. TM info of donor/acceptor chain found but TM info of donor/acceptor residue not found

        if donor_reschain not in list(tm_segments2.keys()): # sometimes the dict "tm_segments2" contains TM segment info corresponding to the auth asym IDs instead of the biopython chain IDs. In such cases, this condition will become true
            flag_donor            = True
            donor_tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to donor chain not found"
        if accep_reschain not in list(tm_segments2.keys()):
            flag_accep = True
            accep_tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to acceptor chain not found"

        if donor_reschain in list(tm_segments2.keys()): 
            flag_donor = True # it will become only when the residue's TM segment info is found
            for segment_type, segment_ranges in tm_segments2[donor_reschain].items():
                for segment_range in segment_ranges:
                    segment_range_start = segment_range[0]
                    segment_range_end   = segment_range[1]
                    if donor_resno in range(segment_range_start, segment_range_end + 1):
                        donor_tm_segment_info = dict_segment_meanings[segment_type]
                        flag_donor            = False
            if flag_donor == True:
                donor_tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to donor residue not found"

        if accep_reschain in list(tm_segments2.keys()):
            flag_accep = True
            for segment_type, segment_ranges in tm_segments2[accep_reschain].items():
                for segment_range in segment_ranges:
                    segment_range_start = segment_range[0]
                    segment_range_end   = segment_range[1]
                    if accep_resno in range(segment_range_start, segment_range_end + 1):
                        accep_tm_segment_info = dict_segment_meanings[segment_type]
                        flag_accep            = False
            if flag_accep == True:
                accep_tm_segment_info = "AAAA_PDB ID found on PDBTM but TM info specific to acceptor residue not found"

    #^###############################################################################
    #^ Code to assign mainchain/sidechain annotations to donor & acceptor atoms:
    #^###############################################################################
    donor_atom_name = desired_donor_atom.get_name()
    accep_atom_name = desired_acceptor_atom.get_name()
    if len(donor_atom_name) == 1 and donor_atom_name in ["O", "N"]:
        donor_atom_location = "Mainchain"
    if len(accep_atom_name) == 1 and accep_atom_name in ["O", "N"]:
        accep_atom_location = "Mainchain"

    if len(donor_atom_name) > 1:
        donor_atom_location = "Sidechain"
    if len(accep_atom_name) > 1:
        accep_atom_location = "Sidechain"

    #^###############################################################################
    #^ Code to classify the hydrogen bond:
    #^###############################################################################
    donor_resno    = desired_donor_atom.get_parent().get_full_id()[3][1]
    accep_resno    = desired_acceptor_atom.get_parent().get_full_id()[3][1]
    if abs(donor_resno - accep_resno) <= 4:
        hbond_type = "Short range SHB"
    if abs(donor_resno - accep_resno) > 9:
        hbond_type = "Long range SHB"
    if abs(donor_resno - accep_resno) in range(5, 9 + 1):
        hbond_type = "Medium range SHB"

    #^################################################################################################################################
    #^ Assign distance & angle cutoffs V1:
    #& (comment it out when finding normal hydrogen bonds in TM dataset and uncomment the "if" statement given in the next section)
    #^################################################################################################################################
    # if desired_donor_atom.get_name()[0] in ["O", "N"]:
    #     DA_cutoff  = 2.7
    #     DHA_cutoff = 150
    # if desired_donor_atom.get_name() == "SG":
    #     DA_cutoff  = 3.5
    #     DHA_cutoff = 150

    #^##########################################################################################################################
    #^ Assign distance & angle cutoffs V2 (sir scolded me for doing fresh analysis using this criteria so I am not using it):
    #^##########################################################################################################################
    # if desired_donor_atom.get_name()[0] in ["O", "N"] and desired_acceptor_atom.get_name()[0] in ["O", "N"]:
    #     DA_cutoff  = 2.7
    #     DHA_cutoff = 150
    # if desired_donor_atom.get_name()[0] in ["O", "N"] and desired_acceptor_atom.get_name()[0] in ["S"]:
    #     DA_cutoff  = 3.5
    #     DHA_cutoff = 150
    # if desired_donor_atom.get_name()[0] in ["S"] and desired_acceptor_atom.get_name()[0] in ["O", "N", "S"]:
    #     DA_cutoff  = 3.5
    #     DHA_cutoff = 150
    if distance_DA > 2.7 and distance_DA <= 3.2 and angle_DHA >= 150: #& use this criteria to identify normal hydrogen bonds in TM dataset (taken from page 8 of Saraswathi mam's paper)
    # if distance_DA <= DA_cutoff and angle_DHA >= DHA_cutoff:
        dictdata.setdefault("Donor atom",                                         []).append(desired_donor_atom.get_name())                          #donor's atom name
        dictdata.setdefault("Donor resname",                                      []).append(desired_donor_atom.get_parent().get_resname())          #donor's residue name
        dictdata.setdefault("Donor resno",                                        []).append(desired_donor_atom.get_parent().get_full_id()[3][1])    #donor's residue no
        dictdata.setdefault("Donor reschain",                                     []).append(desired_donor_atom.get_parent().get_full_id()[2])       #donor's parent chain
        dictdata.setdefault("Donor PDB ID",                                       []).append(pdb_id_molprobity[0:4].upper())                         #donor's parent molecule
        dictdata.setdefault("Donor insertion code",                               []).append(desired_donor_atom.get_parent().get_full_id()[3][2])    #donor's insertion code
        dictdata.setdefault("Acceptor atom",                                      []).append(desired_acceptor_atom.get_name())                       #acceptor's atom name
        dictdata.setdefault("Acceptor resname",                                   []).append(desired_acceptor_atom.get_parent().get_resname())       #acceptor's residue name
        dictdata.setdefault("Acceptor resno",                                     []).append(desired_acceptor_atom.get_parent().get_full_id()[3][1]) #acceptor's residue no
        dictdata.setdefault("Acceptor reschain",                                  []).append(desired_acceptor_atom.get_parent().get_full_id()[2])    #acceptor's parent chain
        dictdata.setdefault("Acceptor PDB ID",                                    []).append(pdb_id_molprobity[0:4].upper())                         #acceptor's parent molecule
        dictdata.setdefault("Acceptor insertion code",                            []).append(desired_acceptor_atom.get_parent().get_full_id()[3][2]) #acceptor's insertion code
        dictdata.setdefault("Donor info",                                         []).append(donor_info)
        dictdata.setdefault("Acceptor info",                                      []).append(accep_info)
        dictdata.setdefault("Donaccep info",                                      []).append(donor_info + "_" + accep_info)

        dictdata.setdefault("H-atom involved",                                    []).append(desired_hydrogen_atom.get_name())                       #if the donor atom is bound to two H-atoms, this will help you know which among the two is engaged in H-bonding
        dictdata.setdefault("DA distance (Å)",                                    []).append(distance_DA)
        dictdata.setdefault("HA distance (Å)",                                    []).append(distance_HA)
        dictdata.setdefault("DHA angle (degrees)",                                []).append(angle_DHA)
        dictdata.setdefault("Occupancy of donor atom",                            []).append(desired_donor_atom.get_occupancy())
        dictdata.setdefault("Occupancy of acceptor atom",                         []).append(desired_acceptor_atom.get_occupancy())
        dictdata.setdefault("Occupancy of hydrogen atom",                         []).append(desired_hydrogen_atom.get_occupancy())
        dictdata.setdefault("Resolution (Å)",                                     []).append(dict_resolutions[pdb_id_molprobity[0:4].upper()])        
        dictdata.setdefault("DA distance in primary sequence",                    []).append(distance_primary_sequence)
        dictdata.setdefault("H-bond type",                                        []).append(hbond_type)
        dictdata.setdefault("Donor residue TM segment info",                      []).append(donor_tm_segment_info)
        dictdata.setdefault("Acceptor residue TM segment info",                   []).append(accep_tm_segment_info)
        dictdata.setdefault("Donor atom location",                                []).append(donor_atom_location)
        dictdata.setdefault("Acceptor atom location",                             []).append(accep_atom_location)
        dictdata.setdefault("Donor & acceptor residues belong to the same chain", []).append(same_chain_or_not)

		#^###############################################################################################################################
		#^ Phi, Psi, Chi1 code:
		#^###############################################################################################################################
        phi_psi_chi1_return_value = ssec_strc_phi_psi_chi1_sasa_value_returner.chimera_phi_psi_chi1_value_returner(desired_donor_atom, desired_acceptor_atom, pdb_id_molprobity, molprobity_files)
        for key, value in phi_psi_chi1_return_value.items(): # it will look like: {"phi" : {"donor" : "None", "acceptor" : "None"}, {"psi" : {"donor" : "None", "acceptor" : "None"}, {"chi1" : {"donor" : "None", "acceptor" : "None"}}
            for acc_don, anglez in value.items():
                dictdata.setdefault(key + " angle " + acc_don + " (degrees)",    []).append(anglez)

#?###############################################################################################################################################################################################
#? Setup the PDB file resolutions, H-atoms bonded to donor atoms, antecedant atoms bonded to acceptor atoms and TM segement info:
#?###############################################################################################################################################################################################
with open("D:\\PhD Work\\Project 03\\files_excel\\SHBs\\molprobity_files_resolutions_tm_dataset.json") as file1:
    dict_resolutions = json.load(file1)
    file1.close()

with open("G:\\My Drive\\python scripts and parameters\\H-bonding parameters.json") as file1:
    dict1 = json.load(file1)
    file1.close()
    dict_donor_and_hydrogen_atoms = {**dict1["dict_donor_and_hydrogen_atoms_ON"], **dict1["dict_donor_and_hydrogen_atoms_S"]}
    dictdata                      = dict()
    dict_no_neighbors             = dict()
    dict_segment_meanings = {"1" : "Extracellular side", 
                             "2" : "Intracellular/Periplasmic side", 
                             "H" : "Transmembrane alpha helix", 
                             "B" : "Transmembrane beta strand", 
                             "C" : "Coiled Structure", 
                             "I" : "Membrane embedded region not interacting with lipids", 
                             "L" : "Membrane embedded region not tranversing the membrane (loop)", 
                             "U" : "Unknown/missing residue", 
                             "F" : "Interfacial helix"}
    relevant_neighbor_dist_cutoff = 4

#^###############################################################################################################################################################################################
#^ Create a dictionary containing PDB IDs and their cooresponding unique chains:
#^###############################################################################################################################################################################################
with open ("D:\\PhD Work\\Project 03\\files_pisces\\results_7\\PISCES_results.txt") as file1:
    list1 = file1.readlines()
    list1 = sorted(list1)
    file1.close()
    dict_id_chains = dict()
    for i in list1:
        dict_id_chains.setdefault(i[0:4], []).append(i[4]) #key will be PDB ID & value will be the no. of distinct chains in that PDB ID
    # print(len(dict_id_chains.keys()))
    # print(dict_id_chains.keys())

#^###############################################################################################################################################################################################
#^ Start identifying the hbonds:
#^###############################################################################################################################################################################################
pdbparser         = PDBParser(PERMISSIVE = True, QUIET = True)
molprobity_files  = "D:\\PhD Work\\Project 03\\files_pdb\\all molprobity files\\CIF+PDB\\"
df_pdbtm          = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\SHBs\\PDBTM segments info.xlsx") # this dataframe stores the info on transmembrane segments

for pdbid, distinct_chains in dict_id_chains.items():
    pdb_id_molprobity = next((molprobity_file_name for molprobity_file_name in os.listdir(molprobity_files) if molprobity_file_name[0:4].upper() == pdbid), None) # "pdb_id_molprobity" will also have extension of the file so that pdbparser can read it from disk
    if pdb_id_molprobity != None:
        print("processing:", pdb_id_molprobity)

        #^##################################################################################################
        #^ Grab the TM annotations of all the distinct chains of the PDB ID:
        #^##################################################################################################
        df_temp     = df_pdbtm[(df_pdbtm["PDB ID"]) == pdbid]
        if len(df_temp) == 0: # it will be true when the PDB ID in the iteration has no corresponding entry in the dataframe e.g., for the PDB IDs "7PJ5", "6ZZ5", "7RF1", "4YUU"
            print("PDB ID not found on PDBTM")
            tm_segments2 = "PDB ID not found on PDBTM" # it will stay empty in this case
        if len(df_temp) != 0:
            row         = df_temp.iloc[0].tolist()     # grab the 1st row of the dataframe (in theory the dataframe should have only one row but just to be safe) & convert to list
            tm_segments = row[4]                       # grab the 5th element of the list as it contains info on tm segments
            if type(tm_segments) == float:             # it will be true when the row corresponding to the PDB ID in the iteration does exist in the excel file but no TM info is available for it (if you print this info, it will be nan)
                print("PDB ID found on PDBTM but TM info not found")
                tm_segments2 = "PDB ID found on PDBTM but TM info not found" # it will stay empty in this case
            if type(tm_segments) != float:            # it will be true only when the TM info is available
                tm_segments  = tm_segments.split(".") # if the PDB file had two chains A & B, then this list will have 3 elements & will look like ["A:71-76(2),77-85(B),86-104(1),105-116(I)", "B:71-76(2),77-85(B),86-104(1)", ""]
                tm_segments  = tm_segments[0:-1]      # the last element of the list is empty so I am removing it
                tm_segments2 = dict()                 # it will contain TM segment info of all the distinct chains of the PDB file in the current iteration; see the last line of this section to know how this dict will look like
                # print(tm_segments)
                for tm_segment in tm_segments:
                    match1 = re.search(r"(\w)(:)(.+)", tm_segment) # it will recognize text like: "A:71-76(2),77-85(B),86-104(1),105-116(I)"
                    if match1:
                        if match1.group(1) in distinct_chains:
                            tm_segments2.setdefault(match1.group(1), {}) # it will look like: {"A" : {}} where "A" is the chain ID
                            list_segments = match1.group(3).split(",")   # it will look like: [71-76(2), 77-85(B), 86-104(1), 105-116(I)]
                            for subsegment in list_segments:
                                match2 = re.search(r"(\d+)(-)(\d+)(\()(\w)(\))", subsegment) # it will recognize text like: "87-104(H)" and "122-130(2)"
                                if match2:
                                    segment_type  = match2.group(5)
                                    segment_start = int(match2.group(1)) 
                                    segment_end   = int(match2.group(3))
                                    tm_segments2[match1.group(1)].setdefault(segment_type, []).append([segment_start, segment_end]) # it will look like: {'A': {'H': [[33, 37], [46, 51]], 'B': [[33, 37], [46, 51]]}}
                # pprint.pprint(tm_segments2)

        #^##################################################################################################
        #^ Continue finding H-bond donors and acceptors:
        #^##################################################################################################
        structure = pdbparser.get_structure("protein", molprobity_files + pdb_id_molprobity)
        residues  = structure.get_residues()
        for residue in residues:
            if residue.get_full_id()[2] in distinct_chains and residue.get_full_id()[3][0] == " " and residue.get_full_id()[3][2] == " ": #if the residue belongs to a unique chain & is a non-heteroatom entity & has no insertion code
                for key, value in dict_donor_and_hydrogen_atoms.items(): # in case a residue has >1 donor atom and if each donor atom is bonded to >1 H-atom, this loop will be required
                    desired_donor_atom = next((res_atom for res_atom in residue.get_atoms() if res_atom.get_name() == key), None)
                    for subvalue in value:
                        desired_hydrogen_atom = next((res_atom for res_atom in residue.get_atoms() if res_atom.get_name() == subvalue), None)
                        if desired_donor_atom is not None and desired_hydrogen_atom is not None:
                            # print(desired_donor_atom, desired_hydrogen_atom)
                            atoms     = list(structure.get_atoms()) # "structure.get_atoms()" is a generator but I had to convert it to a list because iterating over generator throws error
                            neighbors = list()
                            neighbors = NeighborSearch(atoms).search(desired_donor_atom.coord, relevant_neighbor_dist_cutoff)
                            if len(neighbors) == 0:
                                dict_no_neighbors.setdefault(pdb_id_molprobity[0:4], desired_donor_atom.get_parent().get_resname() + str(desired_donor_atom.get_parent().get_full_id()[3][1]) + desired_donor_atom.get_parent().get_full_id()[2])
                            if len(neighbors) != 0:
                                neighbors = [neighbor for neighbor in neighbors if neighbor.get_name()[0] in ["O", "N", "S"]] # I want to keep only those neighbor atoms which are nitrogen, oxygen or sulfur
                                for neighbor in neighbors:
                                    if neighbor.get_parent().get_full_id()[3][0] == " " and neighbor.get_parent().get_full_id()[3][2] == " " and neighbor.get_parent().get_full_id()[2] == desired_donor_atom.get_parent().get_full_id()[2]: # if neighbor atom's parent's residue is a non-heteroatom entity & has no insertion code & belongs to the same chain as the donor atom
                                        desired_acceptor_atom = neighbor
                                        check_flag = False
                                        check_flag = min_dist_checker(dict_donor_and_hydrogen_atoms, desired_donor_atom, desired_acceptor_atom)
                                        if check_flag == True: # it will be True either if the acceptor atom isn't bonded to a H-atom or if it is bonded to a H-atom but the DA distance is less than the distance between D and any H-atom bonded to the acceptor atom
                                            calculator_and_writer(desired_donor_atom, desired_hydrogen_atom, desired_acceptor_atom, pdb_id_molprobity, dictdata, dict_resolutions, molprobity_files, tm_segments2)

print("Neighbors were not found for the following:")
pprint.pprint(dict_no_neighbors)
print("H-bonds finding complete. Post processing started")
df1 = pd.DataFrame(dictdata)
df1["Donor & acceptor atom locations"] = df1["Donor atom location"].astype(str) + '-' + df1["Acceptor atom location"].astype(str) # add another column whose contents will look like: "Mainchain-Mainchain", "Mainchain-Sidechain", and so on

#^###############################################################################################################################################################################################
#^ Take care of bifurcated Hbonds:
#^###############################################################################################################################################################################################
df1.sort_values("Donor PDB ID", inplace = True)
df2                       = df1.copy() #this line causes the code to throw the warning: "A value is trying to be set on a copy of a slice from a DataFrame". It doesn't mean that something is wrong with your code. It simply means that any changes made in "df2" will not affect "df1"
dict_common_acceptor_info = dict()
rows_indices_unwanted_bif = list()
for i in df1.itertuples():
    acceptor_atom     = df1.at[i[0], "Acceptor atom"]
    acceptor_resname  = df1.at[i[0], "Acceptor resname"]
    acceptor_resno    = df1.at[i[0], "Acceptor resno"]
    acceptor_reschain = df1.at[i[0], "Acceptor reschain"]
    acceptor_pdb_id   = df1.at[i[0], "Acceptor PDB ID"]
    tup1              = tuple() + (acceptor_atom, acceptor_resname, acceptor_resno, acceptor_reschain, acceptor_pdb_id) #using tuple and not list because a list cannot become key of a dictionary
    dict_common_acceptor_info.setdefault(tup1, []).append(i[0])                                                         #it will look like: {('O', 'ALA', 52, 'A', '3DB2') [1299, 1300]} for an acceptor which is mentioned in >1 row

for key, value in dict_common_acceptor_info.items():
    if len(value) > 1: #if len(value) > 1, it means that a given acceptor's info is mentioned in >1 row
        dict_donor = dict()
        for i in df1.itertuples():
            if i[0] in value:
                donor_resno    = df1.at[i[0], "Donor resno"]
                donor_reschain = df1.at[i[0], "Donor reschain"]
                donor_pdb_id   = df1.at[i[0], "Donor PDB ID"]
                ha_distance    = df1.at[i[0], "HA distance (Å)"]
                tup2           = tuple() + (donor_resno, donor_reschain, donor_pdb_id)
                dict_donor.setdefault(tup2, []).append(ha_distance)
                # if >1 row has the same acceptor info, it can mean 3 things: 
                    # 1. Each donor instance (an instance is a set of D & H atom that is donating a H-bond) is from a different residue. If this is the case, we will keep all the instances.
                    # 2. Some donors instances atoms are from different residues and some are from the same residue. If this is the case, then we will keep all the instances of the different residues. For the instances belonging to the same residue, there can be 2 subcases:
                        #& 2.1 D1H1 and D2H2 e.g., Arginine's sidechain NH groups. In this case we will keep only the instance with shortest HA distance.
                        #& 2.2 D1H1 and D1H2 e.g., Asparagine's sidechain -NH2 group. In this case also, we will keep only the instance with shortest HA distance.
                        #& In either of the two subcases, the donor's residue info will stay the same & hence their HA distances will append to the same key of the "dict_donor" dictionary

        for key2, value2 in dict_donor.items():
            if len(value2) > 1:                    #if "len(value2) > 1", it means that >1 donor atoms, belonging to the same residue, are donating to the same acceptor atom
                for hadistance in value2:          #in such a case, only keep the instance which is closer to the acceptor atom
                    if hadistance != min(value2):  #if the value in the current iteration is not the smallest value (I am doing this because I want the index number of those rows whose HA distance isn't minimum so that I can delete them later)
                        for j in df2.itertuples(): #then search for it corresponding row in the dataframe & grab its row number
                            donor_resno    = df2.at[j[0], "Donor resno"]
                            donor_reschain = df2.at[j[0], "Donor reschain"]
                            donor_pdb_id   = df2.at[j[0], "Donor PDB ID"]
                            ha_distance    = df2.at[j[0], "HA distance (Å)"]
                            if donor_resno == key2[0] and donor_reschain == key2[1] and donor_pdb_id == key2[2] and ha_distance == hadistance:
                                rows_indices_unwanted_bif.append(j[0])

if len(rows_indices_unwanted_bif) == 0:
    print("No unwanted instances of bifurcated H-bonds found.")
    df3 = pd.DataFrame() #empty dataframe

if len(rows_indices_unwanted_bif) > 0:
    print("Unwanted instances of bifurcated H-bonds found. Their info will be written in a separate tab of the excel file and then their info from the main dataframe will be deleted.")
    df3 = df2.loc[rows_indices_unwanted_bif]
    df3.sort_values("Donor PDB ID", inplace = True)

    for i in rows_indices_unwanted_bif[::-1]: #this loop will do the same thing as "for i in range(len(df1)-1, -1, -1)"; the loop being used right now requires one step less
        df1 = df1.drop([i], axis = 0)
    df1 = df1.reset_index(drop = True)

#^#################################################################################################################
#^Create a datafame where DA occupancy is 1:
#^#################################################################################################################
df4 = df1[(df1["Occupancy of donor atom"] == 1) & (df1["Occupancy of acceptor atom"] == 1)]
df4 = df4.reset_index(drop = True)
df4.sort_values("Donor PDB ID", inplace = True)

#^#################################################################################################################
#^Create a datafame for self contacts:
#^#################################################################################################################
df5 = df4[(df4["Donor resname"] == df4["Acceptor resname"]) & (df4["Donor resno"] == df4["Acceptor resno"]) & (df4["Donor reschain"] == df4["Acceptor reschain"])]
df5 = df5.reset_index(drop = True) #if you don't reset index, then if a self contact was in the 7th row of df4 and it gets added to the 1st row of df5, its index will still stay 7
df5.sort_values("Donor PDB ID", inplace = True)

list_of_worksheets = [
    [df1, "All inst. undesired bif excl."], 
    [df2, "All inst. undesired bif incl."], 
    [df3, "Unwanted bif rows"], 
    [df4, "Instances DA occup one"], 
    [df5, "Self contact instances"], 
    ]

workbook_path = "D:\\Normal H-bonds in TM dataset 2.xlsx" #& use the title "Short H-bonds in TM dataset 2" for SHBs and "Normal H-bonds in TM dataset 2" for normal H-bonds
with pd.ExcelWriter(workbook_path, engine = "xlsxwriter") as writer:
    for df_name, worksheet_name in list_of_worksheets:
        df_name.to_excel(writer, sheet_name = worksheet_name, index = False, na_rep = "NA")
range_to_table.range_to_table(workbook_path)

processing: 1kqfFH.pdb
processing: 1nkzFH.pdb
processing: 1qjpFH.pdb
processing: 1u7gFH.pdb
processing: 1y5iFH.pdb
processing: 1yc9FH.pdb
processing: 2a65FH.pdb
processing: 2ervFH.pdb
processing: 2gr8FH.pdb
processing: 2gufFH.pdb
processing: 2h88FH.pdb
processing: 2jdiFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2ns1FH.pdb
processing: 2onsFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2uuiFH.pdb
processing: 2vqgFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2wjrFH.pdb
processing: 2x55FH.pdb
processing: 2xtvFH.pdb
processing: 2yh5FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2yh9FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 2yhcFH.pdb
PDB ID found on PDBTM but TM info not found
processing: 3aehFH.pdb
processing: 3b9wFH.pdb
processing: 3ck6FH.pdb
PDB ID found on PDBTM but TM info not found
processing: 3fidFH.pdb
processing: 3gp6FH.pdb
processing: 3ldcFH.pdb
processing: 3m73FH.pdb
processing: 3ml3FH.pdb
PD

In [22]:
%reset -f

# Find out the total number of distinct chains which form one SHB, the total number of chains which form two SHBs & so on

First I will find out the total number of SHBs formed by each chain (using a dictionary) & then I will use this dictionary to find out the total number of chains which form 1 SHB, 2SHBs & so on 

In [55]:
%reset -f

In [62]:
import pandas as pd
import numpy as np
from Bio.PDB import *
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import post_processing
import ssec_strc_phi_psi_chi1_sasa_value_returner
import range_to_table
import importlib
importlib.reload(post_processing)
importlib.reload(ssec_strc_phi_psi_chi1_sasa_value_returner)
importlib.reload(range_to_table)

#^###############################################################################################################################################################################################
#^ Create a dictionary containing PDB IDs and their cooresponding unique chains:
#^###############################################################################################################################################################################################
with open ("D:\\My Data\\Study Stuff\\PhD Work\\Project 03\\files_pisces\\results_6\\PISCES_results.txt") as file1:
    list1 = file1.readlines()
    list1 = sorted(list1)
    file1.close()
    dict_id_chains = dict()
    for i in list1:
        dict_id_chains.setdefault(i[0:4] + "_" + i[4], 0) #key will be PDB ID & value will be a counter
    # print(len(dict_id_chains.keys()))
    # print(dict_id_chains.keys())
    # pprint.pprint(dict_id_chains)

#^###############################################################################################################################################################################################
#^ Iterate over the SHB dataframe to find out which chain has how many SHBs:
#^###############################################################################################################################################################################################
df1 = pd.read_excel("E:\\Short H-bonds in TM dataset.xlsx", sheet_name = "Final_dataset")
df1["ID_CHAINS"] = df1["Donor PDB ID"] + "_" + df1["Donor reschain"]
list_id_chains   = list(df1["ID_CHAINS"])
# print(len(list_id_chains))

dict_id_chains_2 = dict_id_chains.copy()
for key, value in dict_id_chains.items():
    a = list_id_chains.count(key)
    dict_id_chains_2[key] = a # it will have key for each PDBID-CHAINID combination & their corresponding values will be the number of hbonds formed in that chain
# print(dict_id_chains_2)
# print(len(dict_id_chains_2))
# print(list(dict_id_chains_2.values()))

#^#########################################################################
#^ Find out the total number of chains with 1 SHB, with 2 SHBs & so on:
#^#########################################################################
dictdata1      = dict()
list_of_values = list(dict_id_chains_2.values())
for i in range(0, max(list_of_values) + 1):
    dictdata1.setdefault(i, list_of_values.count(i))
# pprint.pprint(dictdata1)
# ctr = 0
# for i in dictdata1.keys():
#     if i in range(10, 15 + 1):
#         ctr += 1
# print(ctr)

#^#########################################################################
#^ Modify the above dictionary as per Saraswathi mam's paper's figure:
#^#########################################################################
list_temp      = [i for i in range(0, 10)] + ["10-15", "16-20", "21-25", "26-50", ">50"]
dictdata2      = {i:0 for i in list_temp}

for key, value in dictdata1.items():
    if type(key) == int and key <= 9:
        dictdata2[key] = value
    if key in range(10, 15 + 1):
        dictdata2["10-15"] = dictdata2["10-15"] + value
    if key in range(16, 20 + 1):
        dictdata2["16-20"] = dictdata2["16-20"] + value
    if key in range(21, 25 + 1):
        dictdata2["21-25"] = dictdata2["21-25"] + value
    if key in range(26, 50 + 1):
        dictdata2["26-50"] = dictdata2["26-50"] + value
    if key > 50:
        dictdata2[">50"] = dictdata2[">50"] + value
# pprint.pprint(dictdata2)

dictdata3 = dict()
dictdata3.setdefault("Total number of SHBs", list(dictdata2.keys()))
dictdata3.setdefault("Total number of chains having that many SHBs", list(dictdata2.values()))
df2 = pd.DataFrame(dictdata3)
df2["Total number of SHBs"] = df2["Total number of SHBs"].astype(str)
with pd.ExcelWriter(path = "E:\\Total number of chains having one SHB & so on.xlsx", engine = "xlsxwriter") as writer:
    df2.to_excel(writer, index = False, na_rep = "NA")

In [63]:
%reset -f

# Find out the residues that are neighbors of the donor and acceptor atoms

In [11]:
%reset -f

In [13]:
import pandas as pd
from Bio.PDB import *
import os
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import range_to_table
import importlib
importlib.reload(range_to_table)

pdbparser        = PDBParser(PERMISSIVE = True, QUIET = True)
molprobity_files = "D:\\PhD Work\\Project 03\\files_pdb\\all molprobity files\\CIF+PDB\\"
df1              = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\SHBs\\SHBs in TM dataset 1.xlsx", sheet_name = "Final_dataset")
# df1              = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\SHBs\\SHBs in TM dataset 2.xlsx", sheet_name = "Sheet1")
# df1              = df1[(df1["Donor atom"] == "N") & (df1["Acceptor atom"].isin(["O", "N"]))]
print(len(df1))

dictdata         = dict()
for i in df1.itertuples():
    donor_atom        = df1.at[i[0], "Donor atom"]
    donor_resname     = df1.at[i[0], "Donor resname"]
    donor_resno       = df1.at[i[0], "Donor resno"]
    donor_reschain    = df1.at[i[0], "Donor reschain"]
    donor_resinfo     = df1.at[i[0], "Donor info"]    
    acceptor_atom     = df1.at[i[0], "Acceptor atom"]
    acceptor_resname  = df1.at[i[0], "Acceptor resname"]
    acceptor_resno    = df1.at[i[0], "Acceptor resno"]
    acceptor_reschain = df1.at[i[0], "Acceptor reschain"]
    acceptor_resinfo  = df1.at[i[0], "Acceptor info"] 
    pdb_id            = df1.at[i[0], "Acceptor PDB ID"]
    donor_location    = df1.at[i[0], "Donor atom location"]
    acceptor_location = df1.at[i[0], "Acceptor atom location"]
    print("processing:", donor_resinfo + "_" + acceptor_resinfo)

    #^##########################################################################################################################################################
    #^ Iterate over the residues of the PDB ID in the current iteration & find info about donor & acceptor atoms:
    #^##########################################################################################################################################################
    for pdb_id_molprobity in os.listdir(molprobity_files):
        if pdb_id_molprobity[0:4].upper() == pdb_id:
            structure = pdbparser.get_structure("protein", molprobity_files + pdb_id_molprobity)
            residues  = structure.get_residues()
            for residue in residues:
                if residue.get_resname() == donor_resname and residue.get_full_id()[3][1] == int(donor_resno) and residue.get_full_id()[2] == donor_reschain:
                    atoms = residue.get_atoms()
                    for atom in atoms:
                        if atom.get_name() == donor_atom:
                            desired_donor_atom = atom
                if residue.get_resname() == acceptor_resname and residue.get_full_id()[3][1] == int(acceptor_resno) and residue.get_full_id()[2] == acceptor_reschain:
                    atoms = residue.get_atoms()
                    for atom in atoms:
                        if atom.get_name() == acceptor_atom:
                            desired_acceptor_atom = atom

            #^##########################################################################################################################################################
            #^ Find the atoms lying within 4.5A of donor & acceptor atoms (the value 4.5 is taken from Saraswathi mam's paper):
            #^##########################################################################################################################################################
            atoms     = list(structure.get_atoms()) # "structure.get_atoms()" is a generator but I had to convert it to a list because iterating over generator throws error
            neighbor_atoms_of_donor_atom = NeighborSearch(atoms).search(desired_donor_atom.coord,    4.5)
            neighbor_atoms_of_accep_atom = NeighborSearch(atoms).search(desired_acceptor_atom.coord, 4.5)

            #^##########################################################################################################################################################
            #^ Remove those surrounding atoms which come from the donor or acceptor residue themselves & those which come from mainchain of other residues
            #^ (I am excluding the mainchain atoms of other residues because they don't fit in the definition of "environment" as per Saraswathi mam's paper):
            #^##########################################################################################################################################################
            neighbor_residues_of_donor_atom = dict() # its keys will be neighbor resinfo & values will be lists containing all the atoms that are neighbors of the corresponding neighbor residue
            for neighbor_atom in neighbor_atoms_of_donor_atom:
                neighbor_info     = neighbor_atom.get_parent().get_resname()     + "_" + str(neighbor_atom.get_parent().get_full_id()[3][1]) + "_" + neighbor_atom.get_parent().get_full_id()[2]
                donor_resinfo2    = donor_resname    + "_" + str(donor_resno)    + "_" + donor_reschain
                acceptor_resinfo2 = acceptor_resname + "_" + str(acceptor_resno) + "_" + acceptor_reschain
                if neighbor_info not in [donor_resinfo2, acceptor_resinfo2] and neighbor_atom.get_name() not in ["N", "H", "H1", "H2", "H3", "CA", "HA", "C", "O"]:
                    neighbor_residues_of_donor_atom.setdefault(neighbor_info, []).append(neighbor_atom.get_name())

            neighbor_residues_of_acceptor_atom = dict()
            for neighbor_atom in neighbor_atoms_of_accep_atom:
                neighbor_info     = neighbor_atom.get_parent().get_resname()     + "_" + str(neighbor_atom.get_parent().get_full_id()[3][1]) + "_" + neighbor_atom.get_parent().get_full_id()[2]
                donor_resinfo2    = donor_resname    + "_" + str(donor_resno)    + "_" + donor_reschain
                acceptor_resinfo2 = acceptor_resname + "_" + str(acceptor_resno) + "_" + acceptor_reschain
                if neighbor_info not in [donor_resinfo2, acceptor_resinfo2] and neighbor_atom.get_name() not in ["N", "H", "H1", "H2", "H3", "CA", "HA", "C", "O"]:
                    neighbor_residues_of_acceptor_atom.setdefault(neighbor_info, []).append(neighbor_atom.get_name())

            #^##########################################################################################################################################################
            #^ Count the total number of hydrophobic, aromatic, basic, acidic & polar residues surrounding the donor & acceptor atoms in the current iteration:
            #^##########################################################################################################################################################
            dict_donor_neighbor_residues_counts = {("GLY", "ALA", "VAL", "ILE", "LEU", "MET", "PRO") : 0, ("PHE", "TYR", "TRP") : 0, ("LYS", "ARG", "HIS") : 0, ("ASP", "GLU") : 0, ("CYS", "SER", "THR", "ASN", "GLN") : 0}
            info1 = list()
            for neighbor_residue, neighbor_atoms in neighbor_residues_of_donor_atom.items():
                for key in dict_donor_neighbor_residues_counts.keys():
                    if neighbor_residue[0:3] in key:
                        dict_donor_neighbor_residues_counts[key] += 1
                        for neighbor_atom in neighbor_atoms:
                            info1.append(neighbor_residue + "_" + neighbor_atom)

            dict_acceptor_neighbor_residues_counts = {("GLY", "ALA", "VAL", "ILE", "LEU", "MET", "PRO") : 0, ("PHE", "TYR", "TRP") : 0, ("LYS", "ARG", "HIS") : 0, ("ASP", "GLU") : 0, ("CYS", "SER", "THR", "ASN", "GLN") : 0}
            info2 = list()
            for neighbor_residue, neighbor_atoms in neighbor_residues_of_acceptor_atom.items():
                for key in dict_acceptor_neighbor_residues_counts.keys():
                    if neighbor_residue[0:3] in key:
                        dict_acceptor_neighbor_residues_counts[key] += 1
                        for neighbor_atom in neighbor_atoms:
                            info2.append(neighbor_residue + "_" + neighbor_atom)

            dictdata.setdefault("Donor resinfo",                                            []).append(donor_resinfo)
            dictdata.setdefault("Acceptor resinfo",                                         []).append(acceptor_resinfo)
            dictdata.setdefault("Donor atom location",                                      []).append(donor_location)
            dictdata.setdefault("Acceptor atom location",                                   []).append(acceptor_location)
            dictdata.setdefault("Number of hydrophobic neighbor residues of donor atom",    []).append(dict_donor_neighbor_residues_counts[("GLY", "ALA", "VAL", "ILE", "LEU", "MET", "PRO")])
            dictdata.setdefault("Number of hydrophobic neighbor residues of acceptor atom", []).append(dict_acceptor_neighbor_residues_counts[("GLY", "ALA", "VAL", "ILE", "LEU", "MET", "PRO")])
            dictdata.setdefault("Number of aromatic neighbor residues of donor atom",       []).append(dict_donor_neighbor_residues_counts[("PHE", "TYR", "TRP")])
            dictdata.setdefault("Number of aromatic neighbor residues of acceptor atom",    []).append(dict_acceptor_neighbor_residues_counts[("PHE", "TYR", "TRP")])
            dictdata.setdefault("Number of basic neighbor residues of donor atom",          []).append(dict_donor_neighbor_residues_counts[("LYS", "ARG", "HIS")])
            dictdata.setdefault("Number of basic neighbor residues of acceptor atom",       []).append(dict_acceptor_neighbor_residues_counts[("LYS", "ARG", "HIS")])
            dictdata.setdefault("Number of acidic neighbor residues of donor atom",         []).append(dict_donor_neighbor_residues_counts[("ASP", "GLU")])
            dictdata.setdefault("Number of acidic neighbor residues of acceptor atom",      []).append(dict_acceptor_neighbor_residues_counts[("ASP", "GLU")])
            dictdata.setdefault("Number of polar neighbor residues of donor atom",          []).append(dict_donor_neighbor_residues_counts[("CYS", "SER", "THR", "ASN", "GLN")])
            dictdata.setdefault("Number of polar neighbor residues of acceptor atom",       []).append(dict_acceptor_neighbor_residues_counts[("CYS", "SER", "THR", "ASN", "GLN")])
            dictdata.setdefault("All neighbor residues of donor atom",                      []).append("   ".join(info1))
            dictdata.setdefault("All neighbor residues of acceptor atom",                   []).append("   ".join(info2))

df2           = pd.DataFrame(dictdata)
dictdata2 = {
    "HYDROPHOBIC donor"    : [df2["Number of hydrophobic neighbor residues of donor atom"].sum()], 
    "HYDROPHOBIC acceptor" : [df2["Number of hydrophobic neighbor residues of acceptor atom"].sum()], 
    "AROMATIC donor"       : [df2["Number of aromatic neighbor residues of donor atom"].sum()], 
    "AROMATIC acceptor"    : [df2["Number of aromatic neighbor residues of acceptor atom"].sum()], 
    "BASIC donor"          : [df2["Number of basic neighbor residues of donor atom"].sum()], 
    "BASIC acceptor"       : [df2["Number of basic neighbor residues of acceptor atom"].sum()], 
    "ACIDIC donor"         : [df2["Number of acidic neighbor residues of donor atom"].sum()], 
    "ACIDIC acceptor"      : [df2["Number of acidic neighbor residues of acceptor atom"].sum()], 
    "POLAR donor"          : [df2["Number of polar neighbor residues of donor atom"].sum()], 
    "POLAR acceptor"       : [df2["Number of polar neighbor residues of acceptor atom"].sum()], 
    }

df3 = pd.DataFrame(dictdata2)
workbook_path = "D:\\Neighbors residues of donor & acceptor atoms dataset 1.xlsx"
with pd.ExcelWriter(workbook_path, engine = "xlsxwriter") as writer:
    df2.to_excel(writer, sheet_name = "Neighbor_info",     index = False, na_rep = "NA")
    df3.to_excel(writer, sheet_name = "Neighbor_info_sum", index = False, na_rep = "NA")
range_to_table.range_to_table(workbook_path)

927
processing: 1E54_N_LEU_208_A_1E54_O_LEU_223_A
processing: 1E54_N_THR_5_A_1E54_O_ARG_42_A
processing: 1E54_NH1_ARG_75_A_1E54_OE2_GLU_58_A
processing: 1E54_N_SER_80_A_1E54_O_GLY_54_A
processing: 1E54_OG_SER_103_A_1E54_O_LYS_100_A
processing: 1E54_OH_TYR_289_A_1E54_OD1_ASP_11_A
processing: 1E54_OG1_THR_110_A_1E54_O_SER_32_A
processing: 1E54_N_GLY_244_A_1E54_O_SER_220_A
processing: 1E54_N_ASN_291_A_1E54_O_GLN_325_A
processing: 1E54_OG_SER_220_A_1E54_O_TYR_104_A
processing: 1EHK_OG_SER_109_A_1EHK_O_MET_106_A
processing: 1EHK_OG1_THR_312_A_1EHK_O_PRO_308_A
processing: 1EHK_OG1_THR_302_A_1EHK_O_HIS_298_A
processing: 1EHK_OG_SER_309_A_1EHK_OH_TYR_244_A
processing: 1EHK_OG1_THR_231_A_1EHK_O_ALA_196_A
processing: 1EHK_OG_SER_400_A_1EHK_O_ALA_397_A
processing: 1EK9_OG1_THR_254_A_1EK9_OD1_ASN_282_A
processing: 1H2S_OG1_THR_33_B_1H2S_O_VAL_29_B
processing: 1KB9_OG1_THR_124_C_1KB9_O_ILE_120_C
processing: 1KB9_OH_TYR_106_C_1KB9_OD2_ASP_309_C
processing: 1KB9_OH_TYR_359_C_1KB9_O_ALA_298_C
processi

In [12]:
%reset -f

# Create COCH3-Residue-NHCH3 systems

Before running this script, ensure that the file "chimera_hadding_script_for_short_hydrogen_bonds.py" is in the D drive

In [33]:
%reset -f

In [34]:
from Bio.PDB import *
import pandas as pd
import os
import json
import re

pdbparser             = PDBParser(PERMISSIVE = True, QUIET = True)
path_molprobity_files = "D:\\PhD Work\\Project 03\\files_pdb\\all molprobity files\\CIF+PDB\\"
path_excel_file       = "D:\\PhD Work\\Project 03\\files_excel\\short hydrogen bonds\\SHBs in TM dataset 2.xlsx"
dict_unprocessed      = dict()
dict_unprocessed_2    = dict()
df1                   = pd.read_excel(path_excel_file, sheet_name = "DA occup one")
residue = "SHB"
dict_unprocessed.setdefault(residue,   {"Total Unprocessed residues" : 0})
dict_unprocessed_2.setdefault(residue, {})

desired_n_terminal_neighbors = ["C", "O", "CA", "HA", "HA2"] #GLY residues have "HA2" instead of "HA"
desired_c_terminal_neighbors = ["N", "H", "CA", "HA", "HA2"] #PRO as a C-terminal neighbor will be skipped because the N-atom of PRO is not bonded to a H-atom

for i in df1.itertuples():
    donor_pdb_id   = df1.at[i[0], "Donor PDB ID"]
    donor_atom     = df1.at[i[0], "Donor atom"]
    donor_resname  = df1.at[i[0], "Donor resname"]
    donor_resno    = df1.at[i[0], "Donor resno"]
    donor_reschain = df1.at[i[0], "Donor reschain"]

    accep_atom     = df1.at[i[0], "Acceptor atom"]
    accep_resname  = df1.at[i[0], "Acceptor resname"]
    accep_resno    = df1.at[i[0], "Acceptor resno"]
    accep_reschain = df1.at[i[0], "Acceptor reschain"]

    for pdb_file in os.listdir(path_molprobity_files):
        if pdb_file[0:4].upper() == donor_pdb_id:
            print("processing row:", i[0])
            structure   = pdbparser.get_structure("protein", path_molprobity_files + pdb_file)
            dict_atoms  = {"donor atoms"    : [], "N-terminal neighbor atoms"  : [], "C-terminal neighbor atoms"  : []}
            dict_atoms2 = {"acceptor atoms" : [], "N-terminal neighbor atoms2" : [], "C-terminal neighbor atoms2" : []}
            atoms       = structure.get_atoms()

            for atom in atoms:

                if atom.get_parent().get_full_id()[3][1] == donor_resno and atom.get_parent().get_full_id()[2] == donor_reschain and atom.get_parent().get_resname() == donor_resname:
                    dict_atoms["donor atoms"].append(atom)
                if atom.get_parent().get_full_id()[3][1] == donor_resno - 1 and atom.get_parent().get_full_id()[2] == donor_reschain and atom.get_name() in desired_n_terminal_neighbors:
                    dict_atoms["N-terminal neighbor atoms"].append(atom)
                if atom.get_parent().get_full_id()[3][1] == donor_resno + 1 and atom.get_parent().get_full_id()[2] == donor_reschain and atom.get_name() in desired_c_terminal_neighbors:
                    dict_atoms["C-terminal neighbor atoms"].append(atom)

                if atom.get_parent().get_full_id()[3][1] == accep_resno and atom.get_parent().get_full_id()[2] == accep_reschain and atom.get_parent().get_resname() == accep_resname:
                    dict_atoms2["acceptor atoms"].append(atom)
                if atom.get_parent().get_full_id()[3][1] == accep_resno - 1 and atom.get_parent().get_full_id()[2] == accep_reschain and atom.get_name() in desired_n_terminal_neighbors:
                    dict_atoms2["N-terminal neighbor atoms2"].append(atom)
                if atom.get_parent().get_full_id()[3][1] == accep_resno + 1 and atom.get_parent().get_full_id()[2] == accep_reschain and atom.get_name() in desired_c_terminal_neighbors:
                    dict_atoms2["C-terminal neighbor atoms2"].append(atom)
            
            dict_atoms_donaccep = dict()
            dict_atoms_donaccep = {**dict_atoms, **dict_atoms2} # combine the donor and acceptor dictionaries into one
            #^######################################################################################################################################################################################################################################################################################
            #^ If all the relevant atoms are found, then do the following:
            #^######################################################################################################################################################################################################################################################################################
            if (len(dict_atoms["N-terminal neighbor atoms"]   + dict_atoms["C-terminal neighbor atoms"])   == 8 and                                      # if all the neighbor atoms are found
                len(dict_atoms2["N-terminal neighbor atoms2"] + dict_atoms2["C-terminal neighbor atoms2"]) == 8 and   
                not any(neighbor_atom.get_occupancy() != 1 for neighbor_atom in [atomz for sublist in dict_atoms_donaccep.values() for atomz in sublist])  # kisi bhi atom ki occupancy one se kam nahi honi chahiye
                ):
                path_destination = "D:\\unhadded_files\\"
                if not os.path.isdir(path_destination):
                    os.makedirs(path_destination)

                class Donor_Acceptor_Select(Select): #call the "Donor_Acceptor_Select" class (I arbitrarily chose this name; you can call this class anything you want)
                    def accept_atom(self, atom):     #and within the class, define the "accept_atom()" method (this is a pre-defined biopython method; see pg 171 of biopython cookbook 2018 edition) which takes 2 arguments; self and "atom" which contains info of ALL the atoms
                        if list(atom.get_coord()) in [list(atomz.get_coord()) for sublist in dict_atoms_donaccep.values() for atomz in sublist]:
                            return True
                        else:
                            return False

                io = PDBIO()                # create an instance of the PDBIO class
                io.set_structure(structure) # pass the "structure" variable into "set_structure()" method as argument
                system_path = path_destination + donor_pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + str(donor_reschain) + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + str(accep_reschain) + "_donaccep.pdb"
                io.save(system_path, Donor_Acceptor_Select())

                #^#####################################################################################################################################################################################################################
                #^ If the distance between D & A in primary sequence > 2, then open the created PDB file, remove any pre-existing "TER" lines & add a new "TER" line between the donor lines & acceptor + acceptor's neighbor lines:
                #^#####################################################################################################################################################################################################################
                if abs(donor_resno - accep_resno) > 2:
                    with open(system_path) as file1:
                        list1 = file1.readlines()
                        file1.close()
                    list1          = [i for i in list1 if not i.startswith("TER")] #remove any pre-existing TER rows from the PDB file
                    donor_lines    = {"Nterm_neighbor" :  [], "Cterm_neighbor" :  [], "donor" :    []} # it will store only the lines corresponding to donor atoms & its neighbor atoms
                    accep_lines    = {"Nterm_neighbor2" : [], "Cterm_neighbor2" : [], "acceptor" : []} # it will store only the lines corresponding to acceptor atoms & its neighbor atoms
                    lines_with_ter = list()
                    for line in list1:
                        match1 = re.search(r"(ATOM\s+|HETATM\s+)(\d+)(\s+\w{1,2}\d*)(\s*\w{3,4})(\s)(\w\s*-*\d+)(.+)", line) # "match1.group(1)" has HETATM because if the N-terminal neighbor of acceptor residue is MSE, then its line will start with "HETATM"
                        if match1:
                            reschain_resno = match1.group(6).replace(" ", "") # "match1.group(6)" will contain reschain & resno, either in the format "A 100" or "A100". If there is a space, then remove it
                            reschain       = reschain_resno[0]                # all the chains in the dataset are represented by a single alphabet so be assured that "reschain_resno[0]" denotes the chain ID
                            resno          = reschain_resno[1:]

                            if int(resno) == donor_resno:
                                donor_lines["donor"].append(line)
                            if int(resno) == donor_resno - 1:
                                donor_lines["Nterm_neighbor"].append(line)
                            if int(resno) == donor_resno + 1:
                                donor_lines["Cterm_neighbor"].append(line)

                            if int(resno) == accep_resno:
                                accep_lines["acceptor"].append(line)
                            if int(resno) == accep_resno - 1:
                                accep_lines["Nterm_neighbor2"].append(line)
                            if int(resno) == accep_resno + 1:
                                accep_lines["Cterm_neighbor2"].append(line)

                    if donor_resno < accep_resno:
                        lines_with_ter = donor_lines["Nterm_neighbor"] + donor_lines["donor"] + donor_lines["Cterm_neighbor"] + ["TER\n"] + accep_lines["Nterm_neighbor2"] + accep_lines["acceptor"] + accep_lines["Cterm_neighbor2"] + ["END\n"]

                    if donor_resno > accep_resno:
                        lines_with_ter = accep_lines["Nterm_neighbor2"] + accep_lines["acceptor"] + accep_lines["Cterm_neighbor2"] + ["TER\n"] + donor_lines["Nterm_neighbor"] + donor_lines["donor"] + donor_lines["Cterm_neighbor"] + ["END\n"]

                    with open(system_path, "w") as file2: #overwrite the older system file
                        for line in lines_with_ter:
                            file2.write(line)

            resinfo              = donor_pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + str(donor_reschain) + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + str(accep_reschain)
            flag                 = True  # if it becomes false (any of the condition in the following code can make it happen), it means that the residue in the current cannot be converted to system
            str_conditions_found = str() # if for the residue in the current iteration, the system could not get created, it will store all the conditions that led to such an outcome

            #^######################################################################################################################################################################################################################################################################################
            #^ Suppose for the self-contacting residue in the current iteration, the N-terminal neighbor residue wasn't found (condition A), then the following key will be added to the dictionary: {"condition A" : res1}
            #^ Suppose for 2nd self-contacting residue in the iteration, the N-terminal neighbor residue wasn't found (condition A) as well as the C-terminal neighbor was PRO (condition B) , then the following key will be added to the dictionary: {"condition A + condition B": res2}
            #& Note that although the 2nd residue satisfies condition A as well, its info WILL NOT be added to the condition A key because that's how dictionaries work.  
            #^######################################################################################################################################################################################################################################################################################
            if len(dict_atoms["N-terminal neighbor atoms"]) == 0:
                condition_found      = "N-terminal neighbor residue not found"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if len(dict_atoms["C-terminal neighbor atoms"]) == 0:
                condition_found      = "C-terminal neighbor residue not found"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if len(dict_atoms["N-terminal neighbor atoms"]) > 0 and len(dict_atoms["N-terminal neighbor atoms"]) < 4:
                condition_found      = "Few of the N-terminal neighbor atoms not found"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if len(dict_atoms["C-terminal neighbor atoms"]) > 0 and len(dict_atoms["C-terminal neighbor atoms"]) < 4:
                condition_found      = "Few of the C-terminal neighbor atoms not found"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if any(neighbor_atom.get_occupancy() != 1 for neighbor_atom in dict_atoms["N-terminal neighbor atoms"]):
                condition_found      = "Occupancy of one of the relevant atoms of the N-terminal neighbor residue less than one"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if any(neighbor_atom.get_occupancy() != 1 for neighbor_atom in dict_atoms["C-terminal neighbor atoms"]):
                condition_found      = "Occupancy of one of the relevant atoms of the C-terminal neighbor residue less than one"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if any(neighbor_atom.get_occupancy() != 1 for neighbor_atom in dict_atoms["donor atoms"]):
                condition_found      = "Occupancy of one of the donor atoms less than one"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if any(neighbor_atom.get_parent().get_resname() == "PRO" for neighbor_atom in dict_atoms["C-terminal neighbor atoms"]):
                condition_found      = "C-terminal neighbor residue is PROLINE"
                str_conditions_found = str_conditions_found + condition_found + "+"
                dict_unprocessed[residue].setdefault(resinfo, []).append(condition_found)
                flag = False

            if flag == False:
                dict_unprocessed[residue]["Total Unprocessed residues"] += 1
                dict_unprocessed_2[residue].setdefault(str_conditions_found[0:-1], []).append(resinfo) # "[0:-1]" ensures that the very last "_" from the string is omitted

with open("D:\\Residuewise info of unprocessed examples.json", "w") as file1:
    json.dump(dict_unprocessed, file1, indent = 1)
with open("D:\\Conditionwise info of unprocessed examples.json", "w") as file1:
    json.dump(dict_unprocessed_2, file1, indent = 1)
print("Systems generation complete. Initiating addition of H-atoms to the caps....")

#^########################################################
#^ Add H-atoms to system caps:
#^########################################################
import subprocess
p1 = subprocess.Popen(["C:\\Program Files\\Chimera 1.17.1\\bin\\chimera.exe", "--script", "D:\\chimera_hadding_script_for_short_hydrogen_bonds.py"], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
out, err = p1.communicate()
print(out.decode("utf-8"))
print(err.decode("utf-8"))
print("Addition of H-atoms to the caps complete. Initiating removal of duplicate models (if any) in the system files....")

#^########################################################
#^ Remove duplicate models (if any) in the system files:
#^########################################################
import os
import shutil

path_hadded_files = "D:\\hadded_files\\"
for file in os.listdir(path_hadded_files):
    with open (path_hadded_files + file) as file1:
        list1 = file1.readlines()
        file1.close()

        flag = True
        for i in range(len(list1)):
            if "MODEL" in list1[i]:
                flag = False

        if flag == False:                               # if flag became false, it means that the line containing  "MODEL" was found in the file, indicating that the file contains >1 model and needs processing
            print("file with multiple models:", file)
            os.remove(path_hadded_files + file)                   # delete the file with multiple models
            ctr = 0
            list2 = list()
            for i in range(len(list1)):                 #to better understand what is happening, see the file named "pdb file containing 2 models"
                if "MODEL" in list1[i]:
                    ctr += 1
                if ctr < 2:
                    list2.append(list1[i])

            list2.pop(0)                                #pop the very 1st line of "list2" which contains the string "MODEL"
            for i in range(len(list2)-1, -1, -1):       #iterate over "list2" from reverse & delete the element containing the word "ENDMDL" (if you do not delete it, gaussian will fail to convert it to a gaussian input file)
                if "ENDMDL" in list2[i]:
                    list2.pop(i)
            list2.append("END\n")                       #append a line containing the string "END\n" to the end of "list2" to mark the end of the file
            with open(path_hadded_files + file, "w") as file2:
                for line in list2:
                    file2.write(line)

print("Removal of duplicate models (if any) in the system files complete.")

processing row: 0
processing row: 1
processing row: 2
processing row: 3
processing row: 4
processing row: 5
processing row: 6
processing row: 7
processing row: 8
processing row: 9
processing row: 10
processing row: 11
processing row: 12
processing row: 13
processing row: 14
processing row: 15
processing row: 16
processing row: 17
processing row: 18
processing row: 19
processing row: 20
processing row: 21
processing row: 22
processing row: 23
processing row: 24
processing row: 25
processing row: 26
processing row: 27
processing row: 28
processing row: 29
processing row: 30
processing row: 31
processing row: 32
processing row: 33
processing row: 34
processing row: 35
processing row: 36
processing row: 37
processing row: 38
processing row: 39
processing row: 40
processing row: 41
processing row: 42
processing row: 43
processing row: 44
processing row: 45
processing row: 46
processing row: 47
processing row: 48
processing row: 49
processing row: 50
processing row: 51
processing row: 52
pro

In [10]:
%reset -f

# Create "mainchain-atom-trimmed" versions of <font color="red">SC-SC type</font> COCH3-Residue-NHCH3 systems
Before running this script, ensure that the file "chimera_hadding_script_for_short_hydrogen_bonds.py" is in the D drive

In [36]:
%reset -f

In [48]:
import os
import re
import pandas as pd
import pprint

path  = "D:\\AAAA\\" # it will contain the COCH3-Residue-NHCH3 systems
path2 = "D:\\unhadded_files\\" #& KEEP THE NAME OF THIS FOLDER AS MENTIONED HERE as it will be used as input by the chimera script for adding H-atoms
if not os.path.isdir(path2):
    os.makedirs(path2)

df1 = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\short hydrogen bonds\\SHBs in TM dataset 2.xlsx", sheet_name = "DA occup one")
df1 = df1[(df1["Donor atom location"] == "Sidechain") & (df1["Acceptor atom location"] == "Sidechain")]
# print(len(df1))

for i in df1.itertuples():
    donor_pdb_id   = df1.at[i[0], "Donor PDB ID"]
    donor_atom     = df1.at[i[0], "Donor atom"]
    donor_resname  = df1.at[i[0], "Donor resname"]
    donor_resno    = df1.at[i[0], "Donor resno"]
    donor_reschain = df1.at[i[0], "Donor reschain"]
    accep_atom     = df1.at[i[0], "Acceptor atom"]
    accep_resname  = df1.at[i[0], "Acceptor resname"]
    accep_resno    = df1.at[i[0], "Acceptor resno"]
    accep_reschain = df1.at[i[0], "Acceptor reschain"]
    donaccep_info  = donor_pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + str(donor_reschain) + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + str(accep_reschain) + "_donaccep_hadded.pdb"

    for i in os.listdir(path):
        if i == donaccep_info:
            print("processing", i)
            with open(path + i) as file1:
                list1 = file1.readlines()
                file1.close()

                dict1 = {"donor" : [], "acceptor" : []}
                for line in list1:
                    match1 = re.search(r"(ATOM\s+|HETATM\s+)(\d+)(\s+\w{1,2}\d*)(\s*\w{3,4})(\s)(\w)(\s*-*\d+)(.+)", line) # "match1.group(1)" has HETATM because if the N-terminal neighbor of acceptor residue is MSE, then its line will start with "HETATM"
                    if match1:
                        line_resno = int(match1.group(7).replace(" ", ""))
                        atom_type  = match1.group(3).replace(" ", "")
                        if line_resno == donor_resno and atom_type not in ["CA", "HA", "HA1", "HA2", "N", "H", "C", "O"]:
                            dict1["donor"].append(line)
                        if line_resno == accep_resno and atom_type not in ["CA", "HA", "HA1", "HA2", "N", "H", "C", "O"]:
                            dict1["acceptor"].append(line)
                # pprint.pprint(dict1)

                list2 = list()
                if donor_resno < accep_resno:
                    list2 = dict1["donor"] + ["TER\n"] + dict1["acceptor"] + ["END\n"]
                if donor_resno > accep_resno:
                    list2 = dict1["acceptor"] + ["TER\n"] + dict1["donor"] + ["END\n"]

                with open(path2 + i, "w") as file2:
                    for line2 in list2:
                        file2.write(line2)


#^########################################################
#^ Add H-atoms to systems:
#^########################################################
import subprocess
p1 = subprocess.Popen(["C:\\Program Files\\Chimera 1.17.1\\bin\\chimera.exe", "--script", "D:\\chimera_hadding_script_for_short_hydrogen_bonds.py"], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
out, err = p1.communicate()
print(out.decode("utf-8"))
print(err.decode("utf-8"))
print("Addition of H-atoms to the caps complete. Initiating removal of duplicate models (if any) in the system files....")

#^########################################################
#^ Remove duplicate models (if any) in the system files:
#^########################################################
import os
import shutil

path_hadded_files = "D:\\hadded_files\\"
for file in os.listdir(path_hadded_files):
    with open (path_hadded_files + file) as file1:
        list1 = file1.readlines()
        file1.close()

        flag = True
        for i in range(len(list1)):
            if "MODEL" in list1[i]:
                flag = False

        if flag == False:                               # if flag became false, it means that the line containing  "MODEL" was found in the file, indicating that the file contains >1 model and needs processing
            print("file with multiple models:", file)
            os.remove(path_hadded_files + file)                   # delete the file with multiple models
            ctr = 0
            list2 = list()
            for i in range(len(list1)):                 #to better understand what is happening, see the file named "pdb file containing 2 models"
                if "MODEL" in list1[i]:
                    ctr += 1
                if ctr < 2:
                    list2.append(list1[i])

            list2.pop(0)                                #pop the very 1st line of "list2" which contains the string "MODEL"
            for i in range(len(list2)-1, -1, -1):       #iterate over "list2" from reverse & delete the element containing the word "ENDMDL" (if you do not delete it, gaussian will fail to convert it to a gaussian input file)
                if "ENDMDL" in list2[i]:
                    list2.pop(i)
            list2.append("END\n")                       #append a line containing the string "END\n" to the end of "list2" to mark the end of the file
            with open(path_hadded_files + file, "w") as file2:
                for line in list2:
                    file2.write(line)

print("Removal of duplicate models (if any) in the system files complete.")

processing 6QGW_OG_SER_715_A+_OD1_ASP_740_A_donaccep_hadded.pdb
processing 6TZK_ND1_HIS_914_A+_OD1_ASN_912_A_donaccep_hadded.pdb
processing 7CZ9_OG1_THR_983_A+_OD1_ASP_410_A_donaccep_hadded.pdb
processing 2A65_OH_TYR_107_A+_OD1_ASP_404_A_donaccep_hadded.pdb
processing 6TZK_OH_TYR_1154_A+_OD2_ASP_881_A_donaccep_hadded.pdb
processing 2A65_OH_TYR_47_A+_OE1_GLU_287_A_donaccep_hadded.pdb
processing 4AFK_OG1_THR_49_A+_OD1_ASP_485_A_donaccep_hadded.pdb
processing 3TDS_SG_CYS_168_A+_OD1_ASN_164_A_donaccep_hadded.pdb
processing 6EHD_OH_TYR_248_A+_OE1_GLU_286_A_donaccep_hadded.pdb
processing 4RJW_OG1_THR_85_A+_OD2_ASP_80_A_donaccep_hadded.pdb
processing 5FOK_NE_ARG_207_A+_OE2_GLU_118_A_donaccep_hadded.pdb
processing 4AFK_OG_SER_157_A+_OE1_GLU_155_A_donaccep_hadded.pdb
processing 1QJP_OH_TYR_94_A+_OE1_GLU_140_A_donaccep_hadded.pdb
processing 7Z09_OH_TYR_57_A+_OD2_ASP_212_A_donaccep_hadded.pdb
processing 3V5U_OH_TYR_138_A+_OD1_ASN_104_A_donaccep_hadded.pdb
processing 4AFK_OG_SER_351_A+_OD2_ASP_162

In [47]:
%reset -f

# Create gaussian input files for interaction energy calculations

Its a manual step

# Add BSSE correction to the interaction energy input files

## When the system is CH3CO-RESIDUE-NHCH3

In [80]:
%reset -f

In [81]:
import os
import re

ctr = 0
don_fragment   = "Fragment=1"
acc_fragment   = "Fragment=2"
# path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_CH3CO_RESIDUE_NHCH3\\all\\PBE0_D3_def2TZVP\\"
path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_CH3CO_RESIDUE_NHCH3\\all\\mp2_6311g_2d2p\\"
# path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_CH3CO_RESIDUE_NHCH3\\all\\b3lyp_6311g_2d2p\\"

for gjf_file in os.listdir(path_gjf_files):
	if gjf_file[-4:] == ".gjf" and "_donaccep_unoptimized_int_energy_query" in gjf_file:
		print("processing:", gjf_file)
		ctr += 1
		donor_resno = int(gjf_file.split("_")[3])
		accep_resno = int(gjf_file.split("_")[7])

		donor_resno_reschain            = "ResNum=" + gjf_file.split("_")[3] + "_" + gjf_file.split("_")[4][0]
		N_neighbor_donor_resno_reschain = "ResNum=" + str(donor_resno - 1)   + "_" + gjf_file.split("_")[4][0]
		C_neighbor_donor_resno_reschain = "ResNum=" + str(donor_resno + 1)   + "_" + gjf_file.split("_")[4][0]

		accep_resno_reschain            = "ResNum=" + gjf_file.split("_")[7] + "_" + gjf_file.split("_")[8]
		N_neighbor_accep_resno_reschain = "ResNum=" + str(accep_resno - 1)   + "_" + gjf_file.split("_")[4][0]
		C_neighbor_accep_resno_reschain = "ResNum=" + str(accep_resno + 1)   + "_" + gjf_file.split("_")[4][0]

		# print(donor_resno_reschain)
		with open(path_gjf_files + gjf_file) as file1:
			list1 = file1.readlines()
			list2 = list()
			file1.close()
			for line in list1:
				if "PDBName=" not in line:
					if line[0] == "#":
						line = line[0:-1] + " counterpoise=2" + "\n" # add the "counterpoise" argument
						list2.append(line)
					if line[0] != "#":
						list2.append(line)
				if "PDBName=" in line:
					match1 = re.search(r"(\s)(\w{1})(\()(PDBName=)(\w+)(.+)(\))(.+)(\n)", line) # the code will will replace match1.group(4), match1.group(5) and match1.group(6) with fragment info. The "match1.group(5)" will contain atom name
					if match1:
						if donor_resno_reschain in line or N_neighbor_donor_resno_reschain in line or C_neighbor_donor_resno_reschain in line:
							line = match1.group(1) + match1.group(2) + match1.group(3) + don_fragment + match1.group(7) + match1.group(8) + match1.group(9) # add proper fragment number to the donor and acceptor halves 
							list2.append(line)
						if accep_resno_reschain in line or N_neighbor_accep_resno_reschain in line or C_neighbor_accep_resno_reschain in line:
							line = match1.group(1) + match1.group(2) + match1.group(3) + acc_fragment + match1.group(7) + match1.group(8) + match1.group(9) # add proper fragment number to the donor and acceptor halves 
							list2.append(line)
						# print(line)
		with open(path_gjf_files + gjf_file, "w") as file2:
			for line in list2:
				file2.write(line)
print("Total processed files:", ctr)

processing: 1KQF_N_THR_268_B+_O_GLY_264_B_donaccep_unoptimized_int_energy_query.gjf
processing: 1QJP_NH1_ARG_96_A+_OE1_GLN_78_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1QJP_OH_TYR_8_A+_OE2_GLU_52_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1QJP_OH_TYR_94_A+_OE1_GLU_140_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_NE1_TRP_365_A+_O_LEU_138_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_N_GLY_244_A+_O_ALA_240_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_N_LEU_136_A+_O_VAL_132_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_N_TRP_135_A+_O_PHE_131_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_OG1_THR_111_A+_O_PHE_107_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_OG1_THR_329_A+_O_GLY_325_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_OG1_THR_380_A+_O_LEU_376_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_OG1_THR_51_A+_O_GLN_47_A_donaccep_unoptimized_i

In [64]:
%reset -f

## When the system is "mainchain-atom-trimmed" version of CH3CO-RESIDUE-NHCH3 systems

In [82]:
%reset -f

In [83]:
import os
import re

ctr = 0
don_fragment   = "Fragment=1"
acc_fragment   = "Fragment=2"
# path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_mainchain_atoms_trimmed\\sc_sc_only\\PBE0_D3_def2TZVP\\"
path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_mainchain_atoms_trimmed\\sc_sc_only\\mp2_6311g_2d2p\\"
# path_gjf_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_mainchain_atoms_trimmed\\sc_sc_only\\b3lyp_6311g_2d2p\\"

for gjf_file in os.listdir(path_gjf_files):
	if gjf_file[-4:] == ".gjf" and "_donaccep_unoptimized_int_energy_query" in gjf_file:
		print("processing:", gjf_file)
		ctr += 1
		donor_resno_reschain = "ResNum=" + gjf_file.split("_")[3] + "_" + gjf_file.split("_")[4][0]
		accep_resno_reschain = "ResNum=" + gjf_file.split("_")[7] + "_" + gjf_file.split("_")[8]
		# print(donor_resno_reschain)
		with open(path_gjf_files + gjf_file) as file1:
			list1 = file1.readlines()
			list2 = list()
			file1.close()
			for line in list1:
				if "PDBName=" not in line:
					if line[0] == "#":
						line = line[0:-1] + " counterpoise=2" + "\n" # add the "counterpoise" argument
						list2.append(line)
					if line[0] != "#":
						list2.append(line)
				if "PDBName=" in line:
					match1 = re.search(r"(\s)(\w{1})(\()(PDBName=)(\w+)(.+)(\))(.+)(\n)", line) # the code will will replace match1.group(4), match1.group(5) and match1.group(6) with fragment info. The "match1.group(5)" will contain atom name
					if match1:
						if donor_resno_reschain in line:
							line = match1.group(1) + match1.group(2) + match1.group(3) + don_fragment + match1.group(7) + match1.group(8) + match1.group(9) # add proper fragment number to the donor and acceptor halves 
							list2.append(line)
						if accep_resno_reschain in line:
							line = match1.group(1) + match1.group(2) + match1.group(3) + acc_fragment + match1.group(7) + match1.group(8) + match1.group(9) # add proper fragment number to the donor and acceptor halves 
							list2.append(line)
						# print(line)
		with open(path_gjf_files + gjf_file, "w") as file2:
			for line in list2:
				file2.write(line)
print("Total processed files:", ctr)

processing: 1QJP_NH1_ARG_96_A+_OE1_GLN_78_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1QJP_OH_TYR_8_A+_OE2_GLU_52_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1QJP_OH_TYR_94_A+_OE1_GLU_140_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1U7G_OH_TYR_62_A+_OG1_THR_137_A_donaccep_unoptimized_int_energy_query.gjf
processing: 1YC9_OG_SER_121_A+_OG1_THR_145_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_NE1_TRP_406_A+_OE2_GLU_402_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_OG_SER_356_A+_OD1_ASN_179_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_OH_TYR_107_A+_OD1_ASP_404_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_OH_TYR_169_A+_NE2_HIS_391_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_OH_TYR_47_A+_OE1_GLU_287_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2A65_OH_TYR_489_A+_OG_SER_389_A_donaccep_unoptimized_int_energy_query.gjf
processing: 2ERV_OG_SER_128_A+_NE2_HIS_126_A_donac

In [ ]:
%reset -f

# Calculate interaction energies

In [73]:
%reset -f

In [ ]:
import shutil
import pandas as pd
import os
import re
import json
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters\\")
import range_to_table
import importlib
importlib.reload(range_to_table)

path_log_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_CH3CO_RESIDUE_NHCH3\\all\\PBE0_D3_def2TZVP\\"
# path_log_files = "D:\\qm_systems\\SHBs\\pre_optimized\\system_files_mainchain_atoms_trimmed\\sc_sc_only\\PBE0_D3_def2TZVP\\"

df1 = pd.read_excel("D:\\PhD Work\\Project 03\\files_excel\\short hydrogen bonds\\SHBs in TM dataset 2.xlsx", sheet_name = "DA occup one")
# df1 = df1[(df1["Donor atom location"] == "Sidechain") & (df1["Acceptor atom location"] == "Sidechain")]

#^####################################################################################
#^ CHOOSE THE CORRECT NAME FOR THE OUTPUT EXCEL FILE:
#^####################################################################################
output_excel_file = path_log_files + "Interaction_energies_unoptimized_" + path_log_files.split("\\")[-4] + ".xlsx"

#^####################################################################################
#^ AND PROCEED:
#^####################################################################################
# print(len(df1))
dictdata          = dict()
ctr = 0
# print(len(df1))
for i in df1.itertuples():
    pdb_id           = df1.at[i[0], "Donor PDB ID"]
    chain_id         = df1.at[i[0], "Donor reschain"]
    da_dist          = df1.at[i[0], "DA distance (Å)"]
    ha_dist          = df1.at[i[0], "HA distance (Å)"]
    dha_angle        = df1.at[i[0], "DHA angle (degrees)"]

    don_atom         = df1.at[i[0], "Donor atom"]
    don_resname      = df1.at[i[0], "Donor resname"]
    don_resno        = df1.at[i[0], "Donor resno"]

    acc_atom         = df1.at[i[0], "Acceptor atom"]
    acc_resname      = df1.at[i[0], "Acceptor resname"]
    acc_resno        = df1.at[i[0], "Acceptor resno"]

    resinfo_don      = pdb_id      + "_"  + don_atom    + "_" + don_resname    + "_" + str(don_resno) + "_" + chain_id
    resinfo_acc      = acc_atom    + "_"  + acc_resname + "_" + str(acc_resno) + "_" + chain_id
    resinfo_donaccep = resinfo_don + "+_" + resinfo_acc + "_donaccep_unoptimized_int_energy_query_output.log" #&##################### make changes here
    # print(resinfo_donaccep)

    for file in os.listdir(path_log_files):
        if file == resinfo_donaccep:
            print("processing:", file)
            ctr += 1
            with open(path_log_files + file) as file1:
                listlines = file1.readlines()
                file1.close()

                if " Normal termination of Gaussian 09" not in listlines[-1]:
                    dictdata.setdefault("Resinfo",                                      []).append(resinfo_donaccep.split("_donaccep_")[0])
                    dictdata.setdefault("DA distance (Å) (PRE-OPT)",                    []).append(da_dist)
                    dictdata.setdefault("HA distance (Å) (PRE-OPT)",                    []).append(ha_dist)
                    dictdata.setdefault("DHA angle (degrees) (PRE-OPT)",                []).append(dha_angle)
                    if "_optimized_" in output_excel_file:
                        dictdata.setdefault("Interaction energy (kcal/mol) (POST-OPT)", []).append("Calculation failed")
                    if "_optimized_" not in output_excel_file:
                        dictdata.setdefault("Interaction energy (kcal/mol) (PRE-OPT)",  []).append("Calculation failed")

                if " Normal termination of Gaussian 09" in listlines[-1]:
                    # flag = True #&##################### if you made the mistake of running int energy calculations on Rakesh PC (mistake cuz the log file its gaussian generates somehow reports regex incompatible interaction energies), then uncomment this line and the "if" statement given below 
                    for listline in listlines:
                        match1 = re.search(r"(^\s+)(complexation energy =)(\s+)(.+)(kcal/mole \(corrected\)\n)", listline)
                        if match1:
                            # flag = False
                            int_energy_kcal_mol = match1.group(4) # yes the interaction is already given in kcal/mol in the output file
                            int_energy_kcal_mol = int_energy_kcal_mol.replace(" ", "")
                            int_energy_kcal_mol = float(int_energy_kcal_mol)
                            dictdata.setdefault("Resinfo",                                      []).append(resinfo_donaccep.split("_donaccep_")[0])
                            dictdata.setdefault("DA distance (Å) (PRE-OPT)",                    []).append(da_dist)
                            dictdata.setdefault("HA distance (Å) (PRE-OPT)",                    []).append(ha_dist)
                            dictdata.setdefault("DHA angle (degrees) (PRE-OPT)",                []).append(dha_angle)
                            if "_optimized_" in output_excel_file:
                                dictdata.setdefault("Interaction energy (kcal/mol) (POST-OPT)", []).append(int_energy_kcal_mol)
                            if "_unoptimized_" in output_excel_file:
                                dictdata.setdefault("Interaction energy (kcal/mol) (PRE-OPT)",  []).append(int_energy_kcal_mol)
                    # if flag == True:
                        # shutil.copy(path_log_files + resinfo_don + "+_" + resinfo_acc + "_donaccep_optimized_int_energy_query.gjf", "D:\\AAAA\\" +  resinfo_don + "+_" + resinfo_acc + "_donaccep_optimized_int_energy_query.gjf")
                        # shutil.copy(path_log_files + resinfo_donaccep, "D:\\AAAA\\" + resinfo_donaccep)
# print(ctr)
df2 = pd.DataFrame(dictdata)
# print(len(df2))
with pd.ExcelWriter(path = output_excel_file, engine = "openpyxl") as writer:
    df2.to_excel(writer, index = False, na_rep = "Not Available")
range_to_table.range_to_table(output_excel_file)

In [69]:
%reset -f

# Extract 2nd order perturbation energies (WIP)

In [72]:
%reset -f

In [ ]:
import os
import re
import pprint
import pandas as pd
import sys
sys.path.append("G:\\My Drive\\python scripts and parameters")
import range_to_table
import importlib
importlib.reload(range_to_table)

#^##########################################
#^ Function to write data to dictionary:
#^##########################################
def dict_writer(dict_data, resinfo2, donor_ssec, donor_nbo_info, acceptor_nbo_info, perturbation_energy):
    dict_data.setdefault("Resinfo",                                     []).append(resinfo2)
    dict_data.setdefault("Donor sec. str.",                             []).append(donor_ssec)
    dict_data.setdefault("Donor NBO info",                              []).append(donor_nbo_info)
    dict_data.setdefault("Acceptor NBO info",                           []).append(acceptor_nbo_info)
    if "NBO_optimized_" or "NBO_partially_optimized_" in path_output_excel_file:
        dict_data.setdefault("Second order perturbation energy (kcal/mol) (POST-OPT)", []).append(perturbation_energy)
    if "NBO_unoptimized_" in path_output_excel_file:
        dict_data.setdefault("Second order perturbation energy (kcal/mol) (PRE-OPT)",  []).append(perturbation_energy)

#^######################################################################################################################################################################
#^ Declare the regexes for cases where the acceptor is mainchain O-atom or mainchain N-atom:
#^######################################################################################################################################################################
dict_regex = {
    "CYS": 
    {"N" : [r"(.+)(LP)(\s\(\s+)(1|2)(\)\s)(N)(\s+)(10)(\s+)(/\d+\.\s)(BD\*)(\(\s+)(\d+)(\)\s)(S)(\s+)(1)(\s)(-)(\s)(H)(\s+)(2)(\s+)(\d+\.\d+)(.+\n)", # two different regexes will have to be used because the numbering of the relevant atoms is not consistent among the log files
            r"(.+)(LP)(\s\(\s+)(1|2)(\)\s)(N)(\s+)(7)(\s+)(/\d+\.\s)(BD\*)(\(\s+)(\d+)(\)\s)(S)(\s+)(10)(\s)(-)(\s)(H)(\s+)(11)(\s+)(\d+\.\d+)(.+\n)"]
            }
            }

#^######################################################################################################################################################################
df1                      = pd.read_excel("D:\\My Data\\Study Stuff\\PhD Work\\Project 02 Selenium H-bonds & chalcogen bonds\\20210302\\files_excel\\Hbonds\\Globular H-bonds CYS sidechain is the donor.xlsx", sheet_name = "Instances DA occup one") #&#### choose the correct sheet
df1                      = df1[(df1["Acceptor atom"] == "N") & (df1["DA distance in primary sequence"] != 0)]


# path = "D:\\qm_systems\\SHN_and_OHN_hbonds\\pre_optimized\\system_files_NONSELFCONT_CYS_mainchainN_acetamide_h2s\\all\\b3lyp_6311g_2d2p\\"  #&################ choose the correct path
path = "D:\\qm_systems\\SHN_and_OHN_hbonds\\pre_optimized\\system_files_NONSELFCONT_CYS_mainchainN_acetamide_h2s\\all\\mp2_6311g_2d2p\\"

# path = "D:\\qm_systems\\SHN_and_OHN_hbonds\\partially_optimized\\sidechain_hg_atom_only\\system_files_NONSELFCONT_CYS_mainchainN_acetamide_h2s\\all\\b3lyp_6311g_2d2p\\"
# path = "D:\\qm_systems\\SHN_and_OHN_hbonds\\partially_optimized\\sidechain_hg_atom_only\\system_files_NONSELFCONT_CYS_mainchainN_acetamide_h2s\\all\\mp2_6311g_2d2p\\"

path_output_excel_file   = path + "NBO_unoptimized_" + path.split("\\")[-4] + ".xlsx"  #&################ make changes here
# path_output_excel_file   = path + "NBO_partially_optimized_" + path.split("\\")[-4] + ".xlsx"
# path_output_excel_file   = path + "NBO_optimized_" + path.split("\\")[-4] + ".xlsx"

dict_data                = dict()
list_donor_nbo           = ["N/A"] * len(df1)
list_acceptor_nbo        = ["N/A"] * len(df1)
list_perturbation_energy = ["N/A"] * len(df1)
ctr = 0

for i in df1.itertuples():
    donor_atom    = df1.at[i[0], "Donor atom"]
    accep_atom    = df1.at[i[0], "Acceptor atom"]
    donor_resname = df1.at[i[0], "Donor resname"]
    accep_resname = df1.at[i[0], "Acceptor resname"]
    donor_resno   = df1.at[i[0], "Donor resno"]
    accep_resno   = df1.at[i[0], "Acceptor resno"]
    reschain      = df1.at[i[0], "Donor reschain"]
    pdb_id        = df1.at[i[0], "Donor PDB ID"]
    donor_ssec    = df1.at[i[0], "Donor sec. str."]
    # resinfo       = pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + reschain + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + reschain + "_" + "donaccep_optimized_nbo_query_output.log" #&######## make changes here
    resinfo       = pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + reschain + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + reschain + "_" + "donaccep_unoptimized_nbo_query_output.log"
    resinfo2      = pdb_id + "_" + donor_atom + "_" + donor_resname + "_" + str(donor_resno) + "_" + reschain + "+_" + accep_atom + "_" + accep_resname + "_" + str(accep_resno) + "_" + reschain
    flag1         = True
    # print(resinfo)

    for logfile in os.listdir(path):
        if logfile == resinfo:
            ctr += 1

            flag1 = False
            print("processing", logfile)
            with open(path + logfile) as file1:
                list1 = file1.readlines()
                file1.close()

            #^###############################################################################################################################################################
            #^ Save all the lines that correspond to the relevant interactions between donor and acceptor atoms to a dictionary:
            #^###############################################################################################################################################################
            dict_matches = dict()
            for line in list1:
                match1 = re.search(dict_regex[donor_resname][accep_atom][0], line)
                match2 = re.search(dict_regex[donor_resname][accep_atom][1], line)
                if match1:
                    dict_matches.setdefault(str(match1.group()), float(match1.group(25)))
                if match2:
                    dict_matches.setdefault(str(match2.group()), float(match2.group(25)))
            # pprint.pprint(dict_matches)

            #^###############################################################################################################################################################
            #^ If no relevant interaction is found between the donor and acceptor atoms, then do this:
            #^###############################################################################################################################################################
            if len(dict_matches) == 0:
                donor_nbo_info      = "No relevant interaction found between donor and acceptor atoms"
                acceptor_nbo_info   = donor_nbo_info
                perturbation_energy = donor_nbo_info
                dict_writer(dict_data, resinfo2, donor_ssec, donor_nbo_info, acceptor_nbo_info, perturbation_energy)

            #^#################################################################################################################################################################################
            #^ If relevant interaction was found, then iterate over the above dictionary & grab the relevant info of the interaction with the highest value of 2nd order perturbation energy:
            #^#################################################################################################################################################################################
            if len(dict_matches) != 0:
                for key, value in dict_matches.items():
                    if value == max(dict_matches.values()):
                        match3 = re.search(dict_regex[donor_resname][accep_atom][0], key)
                        match4 = re.search(dict_regex[donor_resname][accep_atom][1], key)
                        if match3:
                            donor_nbo_info      = match3.group(11) + match3.group(13) + " " + match3.group(15) + match3.group(17) + match3.group(19) + match3.group(21) + match3.group(23)
                            acceptor_nbo_info   = match3.group(2)  + match3.group(4)  + " " + match3.group(6)  + match3.group(8)
                            perturbation_energy = float(match3.group(25))
                        if match4:
                            donor_nbo_info      = match4.group(11) + match4.group(13) + " " + match4.group(15) + match4.group(17) + match4.group(19) + match4.group(21) + match4.group(23)
                            acceptor_nbo_info   = match4.group(2)  + match4.group(4)  + " " + match4.group(6)  + match4.group(8)
                            perturbation_energy = float(match4.group(25))
                        dict_writer(dict_data, resinfo2, donor_ssec, donor_nbo_info, acceptor_nbo_info, perturbation_energy)

    # #^######################################################################################################################
    # #^ If the system file was not created in the first place, then do this:
    # #^######################################################################################################################
    # if flag1 == True:
    #     donor_nbo_info      = "System wasn't created"
    #     acceptor_nbo_info   = donor_nbo_info
    #     perturbation_energy = donor_nbo_info
    #     dict_writer(dict_data, resinfo2, donor_ssec, donor_nbo_info, acceptor_nbo_info, perturbation_energy)

with pd.ExcelWriter(path_output_excel_file, engine = "openpyxl") as writer:
    df2 = pd.DataFrame(dict_data)
    df2.to_excel(writer, sheet_name = "Sheet1", index = False, na_rep = "NA")
range_to_table.range_to_table(path_output_excel_file)

In [ ]:
%reset -f